In [1]:
# Add MOSQITO to the Python path
import sys
import os
sys.path.append('..')

# Import numpy
import numpy as np
import pandas as pd
# Import plot function
import matplotlib.pyplot as plt
# Import mosqito functions
#from mosqito.utils import load
from mosqito.sq_metrics import loudness_zwtv, sharpness_din_tv, roughness_dw, pr_ecma_tv, tnr_ecma_tv
from scipy.io import wavfile
from scipy.signal import resample
from maad.spl import pressure2leq, pressure2dBSPL
from maad.util import mean_dB
import librosa

# Import MOSQITO color sheme [Optional]
from mosqito import COLORS

# To get inline plots (specific to Jupyter notebook)
#%matplotlib notebook

In [ ]:
def load(file, wav_calib=None, mat_signal="", mat_fs="", ch=None):
    """Extract the signal and its time axis from .wav or .uff file,
    resample the signal to 48 kHz, and affects its sampling frequency
    and time signal values.

    Parameters
    ----------
    file : string
        string path to the signal file
    wav_calib : float, optional
        Wav file calibration factor [Pa/FS]. Level of the signal in Pa_peak
        corresponding to the full scale of the .wav file. If None, a
        calibration factor of 1 is considered. Default to None.
    mat_signal : string
        in case of a .mat file, name of the signal variable
    mat_fs : string
        in case of a .mat file, name of the sampling frequency variable

    Outputs
    -------
    signal : numpy.array
        time signal values
    fs : integer
        sampling frequency
    """

    # load the .wav file content
    if file[-3:] == "wav" or file[-3:] == "WAV":
        fs, signal = wavfile.read(file)

        # manage multichannel files
        if signal.ndim > 1:
            signal = signal[:, ch] # signal[:, 0] for first channel, signal[:, 1] for second ch

        # calibration factor for the signal to be in Pa
        if wav_calib is None:
            wav_calib = 1
            print("[Info] A calibration of 1 Pa/FS is considered")
        if isinstance(signal[0], np.int16):
            signal = wav_calib * signal / (2**15 - 1)
        elif isinstance(signal[0], np.int32):
            signal = wav_calib * signal / (2**31 - 1)
        elif isinstance(signal[0], np.float):
            signal = wav_calib * signal

    else:
        raise ValueError("""ERROR: only .wav .mat or .uff files are supported""")

    # resample to 48kHz to allow calculation
    if fs != 48000:
        signal = resample(signal, int(48000 * len(signal) / fs))
        fs = 48000

    return signal, fs


# Run iteration to find most appropiate gain that results in correct values



In [ ]:
def cost_function(current_Navg, current_Nmax, current_N05, desired_Navg, desired_Nmax, desired_N05):
    cost=( np.abs(np.abs(desired_Navg)-np.abs(current_Navg))
        +np.abs(np.abs(desired_Nmax)-np.abs(current_Nmax))
        +np.abs(np.abs(desired_N05)-np.abs(current_N05)))
    return cost

cost=100
gain=4.5
N_avg_true=ground_truth[0,1]
N_max_true=ground_truth[0,2]
N_05_true=ground_truth[0,3]
iteration=0
while cost>0.1 and iteration<10:
    print(iteration)
    # LOAD R0001_segment_binaural_44100_1.wav
    path = "../data/soundscapes/R0001_segment_binaural_44100_1.wav"
    sigL_test, fs = load(path, wav_calib=gain, ch=0) #L
    sigR_test, fs = load(path, wav_calib=gain, ch=1) #R

    # Calculate loudness
    #L
    N_l_test, N_spec_l_test, bark_axis_l_test, time_axis_l_test = loudness_zwtv(sigL_test, fs, field_type="free")
    N_avg_l_test=np.mean(N_l_test)
    N_max_l_test=np.max(N_l_test)
    N_05_l_test=np.percentile(N_l_test,95)
    print("loudness L calculated")
    #R
    N_r_test, N_spec_r_test, bark_axis_r_test, time_axis_r_test = loudness_zwtv(sigR_test, fs, field_type="free")
    N_avg_r_test=np.mean(N_r_test)
    N_max_r_test=np.max(N_r_test)
    N_05_r_test=np.percentile(N_r_test,95)
    print("loudness R calculated")

    N_avg_test=(N_avg_r_test+ N_avg_l_test)/2
    N_max_test=(N_max_r_test+ N_max_l_test)/2
    N_05_test=(N_05_r_test+N_05_l_test)/2

    cost=cost_function(N_avg_test, N_max_test, N_05_test, N_avg_true, N_max_true, N_05_true)
    
    print("Current Error ", cost)
    iteration=iteration+1
    gain=gain+0.1

print("Found! Gain is ", gain, " with error ", cost)


# Array of data

In [ ]:
data = [
    [0.759272, 14.8, 22.8, 17.0, 15.8, 15.2, 14.9, 14.7, 14.5, 14.3, 14.2, 14.0, 13.8, 13.6, 1.82, 1.99, 1.9],
    [0.759272, 15.0, 17.6, 16.1, 15.9, 15.6, 15.4, 15.2, 15.0, 14.9, 14.7, 14.5, 14.2, 14.0, 1.78, 1.93, 1.85],
    [1.363361, 18.0, 27.7, 25.1, 22.5, 20.2, 19.3, 18.7, 18.1, 17.6, 16.6, 14.2, 13.2, 12.9, 1.29, 1.59, 1.41],
    [1.363361, 16.3, 38.1, 31.0, 23.2, 19.0, 15.4, 14.6, 13.9, 13.5, 13.2, 12.9, 12.6, 12.4, 1.28, 1.76, 1.42],
    [1.114716, 8.85, 15.5, 11.5, 10.8, 10.3, 9.76, 9.22, 8.72, 7.95, 7.7, 7.48, 7.14, 6.76, 1.12, 1.6, 1.42],
    [1.114716, 7.57, 12.5, 10.4, 9.58, 8.53, 7.76, 7.41, 7.18, 7.04, 6.93, 6.81, 6.67, 6.56, 1.18, 1.41, 1.29],
    [1.041606, 13.1, 16.5, 14.5, 14.2, 13.8, 13.5, 13.3, 13.0, 12.8, 12.6, 12.4, 12.1, 11.9, 1.31, 1.53, 1.4],
    [1.041606, 13.4, 18.2, 15.9, 14.8, 14.1, 13.7, 13.4, 13.1, 12.9, 12.7, 12.5, 12.2, 12.0, 1.36, 1.58, 1.45],
    [1.181325, 6.8, 11.3, 7.52, 7.25, 7.05, 6.93, 6.83, 6.75, 6.68, 6.61, 6.52, 6.41, 6.33, 1.12, 1.55, 1.25],
    [1.181325, 8.05, 12.6, 9.91, 9.19, 8.42, 8.22, 8.06, 7.9, 7.77, 7.62, 7.44, 7.23, 7.07, 1.08, 1.39, 1.15],
    [1.167849, 8.47, 11.8, 9.91, 9.44, 9.04, 8.71, 8.44, 8.28, 8.16, 8.04, 7.9, 7.72, 7.59, 1.05, 1.47, 1.14],
    [1.167849, 12.7, 15.8, 14.2, 14.0, 13.6, 13.4, 13.2, 12.9, 12.7, 12.4, 11.9, 11.1, 10.6, 1.09, 1.32, 1.18],
    [1.117921, 6.9, 11.5, 8.61, 8.18, 7.67, 7.35, 7.06, 6.81, 6.58, 6.32, 6.09, 5.74, 5.5, 1.09, 1.72, 1.25],
    [1.117921, 6.61, 13.3, 8.65, 7.85, 7.27, 6.91, 6.63, 6.4, 6.21, 6.03, 5.85, 5.6, 5.42, 1.1, 1.67, 1.31],
    [1.463234, 9.05, 10.2, 9.58, 9.47, 9.32, 9.21, 9.13, 9.05, 8.97, 8.89, 8.8, 8.67, 8.56, 882, 955, 921],
    [1.463234, 9.02, 11.4, 9.63, 9.46, 9.28, 9.17, 9.08, 9.0, 8.92, 8.83, 8.74, 8.61, 8.51, 0.89, 1.29, 933],
    [0.987588, 14.8, 17.0, 15.5, 15.4, 15.1, 15.0, 14.9, 14.8, 14.7, 14.5, 14.4, 14.3, 14.1, 1.3, 1.45, 1.35],
    [0.987588, 14.9, 19.0, 15.7, 15.5, 15.3, 15.2, 15.1, 14.9, 14.8, 14.7, 14.5, 14.4, 14.2, 1.29, 1.62, 1.34],
    [1.028043, 16.6, 28.6, 21.7, 19.9, 18.4, 17.4, 16.6, 16.0, 15.5, 15.0, 14.5, 13.7, 13.3, 1.29, 2.07, 1.76],
    [1.028043, 17.0, 26.6, 20.4, 19.3, 18.3, 17.8, 17.3, 16.8, 16.4, 16.0, 15.4, 14.8, 14.2, 1.29, 1.95, 1.64]
]#    gain     Navg  Nmax  N05   N10   N20   N30   N40    N50  N60   N70   N80   N90   N94   Savg  Smax  S05


ground_truth=np.array(data)

# Comparing different non-augmented soundscapes calculation VS ground-truth - loudness

Load signals

In [ ]:

# R0001_segment_binaural_44100_1.wav
path = "../data/soundscapes/R0001_segment_binaural_44100_1.wav"
sigL_0001_1, fs = load(path, wav_calib=4.6*ground_truth[0,0]/ground_truth[0,0], ch=0) #L
sigR_0001_1, fs = load(path, wav_calib=4.6*ground_truth[0,0]/ground_truth[0,0], ch=1) #R

# R0001_segment_binaural_44100_2.wav
path = "../data/soundscapes/R0001_segment_binaural_44100_2.wav"
sigL_0001_2, fs = load(path, wav_calib=4.6*ground_truth[1,0]/ground_truth[0,0], ch=0) #L
sigR_0001_2, fs = load(path, wav_calib=4.6*ground_truth[1,0]/ground_truth[0,0], ch=1) #R

# R0002_segment_binaural_44100_1.wav
path = "../data/soundscapes/R0002_segment_binaural_44100_1.wav"
sigL_0002_1, fs = load(path, wav_calib=4.6*ground_truth[2,0]/ground_truth[0,0], ch=0) #L
sigR_0002_1, fs = load(path, wav_calib=4.6*ground_truth[2,0]/ground_truth[0,0], ch=1) #R

# R0002_segment_binaural_44100_2.wav
path = "../data/soundscapes/R0002_segment_binaural_44100_2.wav"
sigL_0002_2, fs = load(path, wav_calib=4.6*ground_truth[3,0]/ground_truth[0,0], ch=0) #L
sigR_0002_2, fs = load(path, wav_calib=4.6*ground_truth[3,0]/ground_truth[0,0], ch=1) #R

# R0003_segment_binaural_44100_1.wav
path = "../data/soundscapes/R0003_segment_binaural_44100_1.wav"
sigL_0003_1, fs = load(path, wav_calib=4.6*ground_truth[4,0]/ground_truth[0,0], ch=0) #L
sigR_0003_1, fs = load(path, wav_calib=4.6*ground_truth[4,0]/ground_truth[0,0], ch=1) #R

# R0003_segment_binaural_44100_2.wav
path = "../data/soundscapes/R0003_segment_binaural_44100_2.wav"
sigL_0003_2, fs = load(path, wav_calib=4.6*ground_truth[5,0]/ground_truth[0,0], ch=0) #L
sigR_0003_2, fs = load(path, wav_calib=4.6*ground_truth[5,0]/ground_truth[0,0], ch=1) #R

# R0004_segment_binaural_44100_1.wav
path = "../data/soundscapes/R0004_segment_binaural_44100_1.wav"
sigL_0004_1, fs = load(path, wav_calib=4.6*ground_truth[6,0]/ground_truth[0,0], ch=0) #L
sigR_0004_1, fs = load(path, wav_calib=4.6*ground_truth[6,0]/ground_truth[0,0], ch=1) #R

# R0004_segment_binaural_44100_2.wav
path = "../data/soundscapes/R0004_segment_binaural_44100_2.wav"
sigL_0004_2, fs = load(path, wav_calib=4.6*ground_truth[7,0]/ground_truth[0,0], ch=0) #L
sigR_0004_2, fs = load(path, wav_calib=4.6*ground_truth[7,0]/ground_truth[0,0], ch=1) #R

# R0005_segment_binaural_44100_1.wav
path = "../data/soundscapes/R0005_segment_binaural_44100_1.wav"
sigL_0005_1, fs = load(path, wav_calib=4.6*ground_truth[8,0]/ground_truth[0,0], ch=0) #L
sigR_0005_1, fs = load(path, wav_calib=4.6*ground_truth[8,0]/ground_truth[0,0], ch=1) #R

# R0005_segment_binaural_44100_2.wav
path = "../data/soundscapes/R0005_segment_binaural_44100_2.wav"
sigL_0005_2, fs = load(path, wav_calib=4.6*ground_truth[9,0]/ground_truth[0,0], ch=0) #L
sigR_0005_2, fs = load(path, wav_calib=4.6*ground_truth[9,0]/ground_truth[0,0], ch=1) #R

# R0006_segment_binaural_44100_1.wav
path = "../data/soundscapes/R0006_segment_binaural_44100_1.wav"
sigL_0006_1, fs = load(path, wav_calib=4.6*ground_truth[10,0]/ground_truth[0,0], ch=0) #L
sigR_0006_1, fs = load(path, wav_calib=4.6*ground_truth[10,0]/ground_truth[0,0], ch=1) #R

# R0006_segment_binaural_44100_2.wav
path = "../data/soundscapes/R0006_segment_binaural_44100_2.wav"
sigL_0006_2, fs = load(path, wav_calib=4.6*ground_truth[11,0]/ground_truth[0,0], ch=0) #L
sigR_0006_2, fs = load(path, wav_calib=4.6*ground_truth[11,0]/ground_truth[0,0], ch=1) #R

# R0007_segment_binaural_44100_1.wav
path = "../data/soundscapes/R0007_segment_binaural_44100_1.wav"
sigL_0007_1, fs = load(path, wav_calib=4.6*ground_truth[12,0]/ground_truth[0,0], ch=0) #L
sigR_0007_1, fs = load(path, wav_calib=4.6*ground_truth[12,0]/ground_truth[0,0], ch=1) #R

# R0007_segment_binaural_44100_2.wav
path = "../data/soundscapes/R0007_segment_binaural_44100_2.wav"
sigL_0007_2, fs = load(path, wav_calib=4.6*ground_truth[13,0]/ground_truth[0,0], ch=0) #L
sigR_0007_2, fs = load(path, wav_calib=4.6*ground_truth[13,0]/ground_truth[0,0], ch=1) #R

# R0008_segment_binaural_44100_1.wav
path = "../data/soundscapes/R0008_segment_binaural_44100_1.wav"
sigL_0008_1, fs = load(path, wav_calib=4.6*ground_truth[14,0]/ground_truth[0,0], ch=0) #L
sigR_0008_1, fs = load(path, wav_calib=4.6*ground_truth[14,0]/ground_truth[0,0], ch=1) #R

# R0008_segment_binaural_44100_2.wav
path = "../data/soundscapes/R0008_segment_binaural_44100_2.wav"
sigL_0008_2, fs = load(path, wav_calib=4.6*ground_truth[15,0]/ground_truth[0,0], ch=0) #L
sigR_0008_2, fs = load(path, wav_calib=4.6*ground_truth[15,0]/ground_truth[0,0], ch=1) #R

# R0009_segment_binaural_44100_1.wav
path = "../data/soundscapes/R0009_segment_binaural_44100_1.wav"
sigL_0009_1, fs = load(path, wav_calib=4.6*ground_truth[16,0]/ground_truth[0,0], ch=0) #L
sigR_0009_1, fs = load(path, wav_calib=4.6*ground_truth[16,0]/ground_truth[0,0], ch=1) #R

# R0009_segment_binaural_44100_2.wav
path = "../data/soundscapes/R0009_segment_binaural_44100_2.wav"
sigL_0009_2, fs = load(path, wav_calib=4.6*ground_truth[17,0]/ground_truth[0,0], ch=0) #L
sigR_0009_2, fs = load(path, wav_calib=4.6*ground_truth[17,0]/ground_truth[0,0], ch=1) #R

# R00010_segment_binaural_44100_1.wav
path = "../data/soundscapes/R0010_segment_binaural_44100_1.wav"
sigL_0010_1, fs = load(path, wav_calib=4.6*ground_truth[18,0]/ground_truth[0,0], ch=0) #L
sigR_0010_1, fs = load(path, wav_calib=4.6*ground_truth[18,0]/ground_truth[0,0], ch=1) #R

# R0010_segment_binaural_44100_2.wav
path = "../data/soundscapes/R0010_segment_binaural_44100_2.wav"
sigL_0010_2, fs = load(path, wav_calib=4.6*ground_truth[19,0]/ground_truth[0,0], ch=0) #L
sigR_0010_2, fs = load(path, wav_calib=4.6*ground_truth[19,0]/ground_truth[0,0], ch=1) #R

Calculate loudness

In [ ]:
# R0001_segment_binaural_44100_1.wav
#L
N_l_0001_1, N_spec_l_0001_1, bark_axis_l_0001_1, time_axis_l_0001_1 = loudness_zwtv(sigL_0001_1, fs, field_type="free")
N_avg_l_0001_1=np.mean(N_l_0001_1)
N_max_l_0001_1=np.max(N_l_0001_1)
N_05_l_0001_1=np.percentile(N_l_0001_1,95)
print(N_l_0001_1.shape, N_spec_l_0001_1.shape)
#R
N_r_0001_1, N_spec_r_0001_1, bark_axis_r_0001_1, time_axis_r_0001_1 = loudness_zwtv(sigR_0001_1, fs, field_type="free")
N_avg_r_0001_1=np.mean(N_r_0001_1)
N_max_r_0001_1=np.max(N_r_0001_1)
N_05_r_0001_1=np.percentile(N_r_0001_1,95)

# R0001_segment_binaural_44100_2.wav
#L
N_l_0001_2, N_spec_l_0001_2, bark_axis_l_0001_2, time_axis_l_0001_2 = loudness_zwtv(sigL_0001_2, fs, field_type="free")
N_avg_l_0001_2=np.mean(N_l_0001_2)
N_max_l_0001_2=np.max(N_l_0001_2)
N_05_l_0001_2=np.percentile(N_l_0001_2,95)
#R
N_r_0001_2, N_spec_r_0001_2, bark_axis_r_0001_2, time_axis_r_0001_2 = loudness_zwtv(sigR_0001_2, fs, field_type="free")
N_avg_r_0001_2=np.mean(N_r_0001_2)
N_max_r_0001_2=np.max(N_r_0001_2)
N_05_r_0001_2=np.percentile(N_r_0001_2,95)

# R0002_segment_binaural_44100_1.wav
#L
N_l_0002_1, N_spec_l_0002_1, bark_axis_l_0002_1, time_axis_l_0002_1 = loudness_zwtv(sigL_0002_1, fs, field_type="free")
N_avg_l_0002_1=np.mean(N_l_0002_1)
N_max_l_0002_1=np.max(N_l_0002_1)
N_05_l_0002_1=np.percentile(N_l_0002_1,95)
#R
N_r_0002_1, N_spec_r_0002_1, bark_axis_r_0002_1, time_axis_r_0002_1 = loudness_zwtv(sigR_0002_1, fs, field_type="free")
N_avg_r_0002_1=np.mean(N_r_0002_1)
N_max_r_0002_1=np.max(N_r_0002_1)
N_05_r_0002_1=np.percentile(N_r_0002_1,95)

# R0002_segment_binaural_44100_2.wav
#L
N_l_0002_2, N_spec_l_0002_2, bark_axis_l_0002_2, time_axis_l_0002_2 = loudness_zwtv(sigL_0002_2, fs, field_type="free")
N_avg_l_0002_2=np.mean(N_l_0002_2)
N_max_l_0002_2=np.max(N_l_0002_2)
N_05_l_0002_2=np.percentile(N_l_0002_2,95)
#R
N_r_0002_2, N_spec_r_0002_2, bark_axis_r_0002_2, time_axis_r_0002_2 = loudness_zwtv(sigR_0002_2, fs, field_type="free")
N_avg_r_0002_2=np.mean(N_r_0002_2)
N_max_r_0002_2=np.max(N_r_0002_2)
N_05_r_0002_2=np.percentile(N_r_0002_2,95)

# R0003_segment_binaural_44100_1.wav
#L
N_l_0003_1, N_spec_l_0003_1, bark_axis_l_0003_1, time_axis_l_0003_1 = loudness_zwtv(sigL_0003_1, fs, field_type="free")
N_avg_l_0003_1=np.mean(N_l_0003_1)
N_max_l_0003_1=np.max(N_l_0003_1)
N_05_l_0003_1=np.percentile(N_l_0003_1,95)
#R
N_r_0003_1, N_spec_r_0003_1, bark_axis_r_0003_1, time_axis_r_0003_1 = loudness_zwtv(sigR_0003_1, fs, field_type="free")
N_avg_r_0003_1=np.mean(N_r_0003_1)
N_max_r_0003_1=np.max(N_r_0003_1)
N_05_r_0003_1=np.percentile(N_r_0003_1,95)

# R0003_segment_binaural_44100_2.wav
#L
N_l_0003_2, N_spec_l_0003_2, bark_axis_l_0003_2, time_axis_l_0003_2 = loudness_zwtv(sigL_0003_2, fs, field_type="free")
N_avg_l_0003_2=np.mean(N_l_0003_2)
N_max_l_0003_2=np.max(N_l_0003_2)
N_05_l_0003_2=np.percentile(N_l_0003_2,95)
#R
N_r_0003_2, N_spec_r_0003_2, bark_axis_r_0003_2, time_axis_r_0003_2 = loudness_zwtv(sigR_0003_2, fs, field_type="free")
N_avg_r_0003_2=np.mean(N_r_0003_2)
N_max_r_0003_2=np.max(N_r_0003_2)
N_05_r_0003_2=np.percentile(N_r_0003_2,95)

# R0004_segment_binaural_44100_1.wav
#L
N_l_0004_1, N_spec_l_0004_1, bark_axis_l_0004_1, time_axis_l_0004_1 = loudness_zwtv(sigL_0004_1, fs, field_type="free")
N_avg_l_0004_1=np.mean(N_l_0004_1)
N_max_l_0004_1=np.max(N_l_0004_1)
N_05_l_0004_1=np.percentile(N_l_0004_1,95)
#R
N_r_0004_1, N_spec_r_0004_1, bark_axis_r_0004_1, time_axis_r_0004_1 = loudness_zwtv(sigR_0004_1, fs, field_type="free")
N_avg_r_0004_1=np.mean(N_r_0004_1)
N_max_r_0004_1=np.max(N_r_0004_1)
N_05_r_0004_1=np.percentile(N_r_0004_1,95)

# R0004_segment_binaural_44100_2.wav
#L
N_l_0004_2, N_spec_l_0004_2, bark_axis_l_0004_2, time_axis_l_0004_2 = loudness_zwtv(sigL_0004_2, fs, field_type="free")
N_avg_l_0004_2=np.mean(N_l_0004_2)
N_max_l_0004_2=np.max(N_l_0004_2)
N_05_l_0004_2=np.percentile(N_l_0004_2,95)
#R
N_r_0004_2, N_spec_r_0004_2, bark_axis_r_0004_2, time_axis_r_0004_2 = loudness_zwtv(sigR_0004_2, fs, field_type="free")
N_avg_r_0004_2=np.mean(N_r_0004_2)
N_max_r_0004_2=np.max(N_r_0004_2)
N_05_r_0004_2=np.percentile(N_r_0004_2,95)

# R0005_segment_binaural_44100_1.wav
#L
N_l_0005_1, N_spec_l_0005_1, bark_axis_l_0005_1, time_axis_l_0005_1 = loudness_zwtv(sigL_0005_1, fs, field_type="free")
N_avg_l_0005_1=np.mean(N_l_0005_1)
N_max_l_0005_1=np.max(N_l_0005_1)
N_05_l_0005_1=np.percentile(N_l_0005_1,95)
#R
N_r_0005_1, N_spec_r_0005_1, bark_axis_r_0005_1, time_axis_r_0005_1 = loudness_zwtv(sigR_0005_1, fs, field_type="free")
N_avg_r_0005_1=np.mean(N_r_0005_1)
N_max_r_0005_1=np.max(N_r_0005_1)
N_05_r_0005_1=np.percentile(N_r_0005_1,95)

# R0005_segment_binaural_44100_2.wav
#L
N_l_0005_2, N_spec_l_0005_2, bark_axis_l_0005_2, time_axis_l_0005_2 = loudness_zwtv(sigL_0005_2, fs, field_type="free")
N_avg_l_0005_2=np.mean(N_l_0005_2)
N_max_l_0005_2=np.max(N_l_0005_2)
N_05_l_0005_2=np.percentile(N_l_0005_2,95)
#R
N_r_0005_2, N_spec_r_0005_2, bark_axis_r_0005_2, time_axis_r_0005_2 = loudness_zwtv(sigR_0005_2, fs, field_type="free")
N_avg_r_0005_2=np.mean(N_r_0005_2)
N_max_r_0005_2=np.max(N_r_0005_2)
N_05_r_0005_2=np.percentile(N_r_0005_2,95)

# R0006_segment_binaural_44100_1.wav
#L
N_l_0006_1, N_spec_l_0006_1, bark_axis_l_0006_1, time_axis_l_0006_1 = loudness_zwtv(sigL_0006_1, fs, field_type="free")
N_avg_l_0006_1=np.mean(N_l_0006_1)
N_max_l_0006_1=np.max(N_l_0006_1)
N_05_l_0006_1=np.percentile(N_l_0006_1,95)
#R
N_r_0006_1, N_spec_r_0006_1, bark_axis_r_0006_1, time_axis_r_0006_1 = loudness_zwtv(sigR_0006_1, fs, field_type="free")
N_avg_r_0006_1=np.mean(N_r_0006_1)
N_max_r_0006_1=np.max(N_r_0006_1)
N_05_r_0006_1=np.percentile(N_r_0006_1,95)

# R0006_segment_binaural_44100_2.wav
#L
N_l_0006_2, N_spec_l_0006_2, bark_axis_l_0006_2, time_axis_l_0006_2 = loudness_zwtv(sigL_0006_2, fs, field_type="free")
N_avg_l_0006_2=np.mean(N_l_0006_2)
N_max_l_0006_2=np.max(N_l_0006_2)
N_05_l_0006_2=np.percentile(N_l_0006_2,95)
#R
N_r_0006_2, N_spec_r_0006_2, bark_axis_r_0006_2, time_axis_r_0006_2 = loudness_zwtv(sigR_0006_2, fs, field_type="free")
N_avg_r_0006_2=np.mean(N_r_0006_2)
N_max_r_0006_2=np.max(N_r_0006_2)
N_05_r_0006_2=np.percentile(N_r_0006_2,95)

# R0007_segment_binaural_44100_1.wav
#L
N_l_0007_1, N_spec_l_0007_1, bark_axis_l_0007_1, time_axis_l_0007_1 = loudness_zwtv(sigL_0007_1, fs, field_type="free")
N_avg_l_0007_1=np.mean(N_l_0007_1)
N_max_l_0007_1=np.max(N_l_0007_1)
N_05_l_0007_1=np.percentile(N_l_0007_1,95)
#R
N_r_0007_1, N_spec_r_0007_1, bark_axis_r_0007_1, time_axis_r_0007_1 = loudness_zwtv(sigR_0007_1, fs, field_type="free")
N_avg_r_0007_1=np.mean(N_r_0007_1)
N_max_r_0007_1=np.max(N_r_0007_1)
N_05_r_0007_1=np.percentile(N_r_0007_1,95)

# R0007_segment_binaural_44100_2.wav
#L
N_l_0007_2, N_spec_l_0007_2, bark_axis_l_0007_2, time_axis_l_0007_2 = loudness_zwtv(sigL_0007_2, fs, field_type="free")
N_avg_l_0007_2=np.mean(N_l_0007_2)
N_max_l_0007_2=np.max(N_l_0007_2)
N_05_l_0007_2=np.percentile(N_l_0007_2,95)
#R
N_r_0007_2, N_spec_r_0007_2, bark_axis_r_0007_2, time_axis_r_0007_2 = loudness_zwtv(sigR_0007_2, fs, field_type="free")
N_avg_r_0007_2=np.mean(N_r_0007_2)
N_max_r_0007_2=np.max(N_r_0007_2)
N_05_r_0007_2=np.percentile(N_r_0007_2,95)

# R0008_segment_binaural_44100_1.wav
#L
N_l_0008_1, N_spec_l_0008_1, bark_axis_l_0008_1, time_axis_l_0008_1 = loudness_zwtv(sigL_0008_1, fs, field_type="free")
N_avg_l_0008_1=np.mean(N_l_0008_1)
N_max_l_0008_1=np.max(N_l_0008_1)
N_05_l_0008_1=np.percentile(N_l_0008_1,95)
#R
N_r_0008_1, N_spec_r_0008_1, bark_axis_r_0008_1, time_axis_r_0008_1 = loudness_zwtv(sigR_0008_1, fs, field_type="free")
N_avg_r_0008_1=np.mean(N_r_0008_1)
N_max_r_0008_1=np.max(N_r_0008_1)
N_05_r_0008_1=np.percentile(N_r_0008_1,95)

# R0008_segment_binaural_44100_2.wav
#L
N_l_0008_2, N_spec_l_0008_2, bark_axis_l_0008_2, time_axis_l_0008_2 = loudness_zwtv(sigL_0008_2, fs, field_type="free")
N_avg_l_0008_2=np.mean(N_l_0008_2)
N_max_l_0008_2=np.max(N_l_0008_2)
N_05_l_0008_2=np.percentile(N_l_0008_2,95)
#R
N_r_0008_2, N_spec_r_0008_2, bark_axis_r_0008_2, time_axis_r_0008_2 = loudness_zwtv(sigR_0008_2, fs, field_type="free")
N_avg_r_0008_2=np.mean(N_r_0008_2)
N_max_r_0008_2=np.max(N_r_0008_2)
N_05_r_0008_2=np.percentile(N_r_0008_2,95)

# R0009_segment_binaural_44100_1.wav
#L
N_l_0009_1, N_spec_l_0009_1, bark_axis_l_0009_1, time_axis_l_0009_1 = loudness_zwtv(sigL_0009_1, fs, field_type="free")
N_avg_l_0009_1=np.mean(N_l_0009_1)
N_max_l_0009_1=np.max(N_l_0009_1)
N_05_l_0009_1=np.percentile(N_l_0009_1,95)
#R
N_r_0009_1, N_spec_r_0009_1, bark_axis_r_0009_1, time_axis_r_0009_1 = loudness_zwtv(sigR_0009_1, fs, field_type="free")
N_avg_r_0009_1=np.mean(N_r_0009_1)
N_max_r_0009_1=np.max(N_r_0009_1)
N_05_r_0009_1=np.percentile(N_r_0009_1,95)

# R0009_segment_binaural_44100_2.wav
#L
N_l_0009_2, N_spec_l_0009_2, bark_axis_l_0009_2, time_axis_l_0009_2 = loudness_zwtv(sigL_0009_2, fs, field_type="free")
N_avg_l_0009_2=np.mean(N_l_0009_2)
N_max_l_0009_2=np.max(N_l_0009_2)
N_05_l_0009_2=np.percentile(N_l_0009_2,95)
#R
N_r_0009_2, N_spec_r_0009_2, bark_axis_r_0009_2, time_axis_r_0009_2 = loudness_zwtv(sigR_0009_2, fs, field_type="free")
N_avg_r_0009_2=np.mean(N_r_0009_2)
N_max_r_0009_2=np.max(N_r_0009_2)
N_05_r_0009_2=np.percentile(N_r_0009_2,95)

# R00010_segment_binaural_44100_1.wav
#L
N_l_0010_1, N_spec_l_0010_1, bark_axis_l_0010_1, time_axis_l_0010_1 = loudness_zwtv(sigL_0010_1, fs, field_type="free")
N_avg_l_0010_1=np.mean(N_l_0010_1)
N_max_l_0010_1=np.max(N_l_0010_1)
N_05_l_0010_1=np.percentile(N_l_0010_1,95)
#R
N_r_0010_1, N_spec_r_0010_1, bark_axis_r_0010_1, time_axis_r_0010_1 = loudness_zwtv(sigR_0010_1, fs, field_type="free")
N_avg_r_0010_1=np.mean(N_r_0010_1)
N_max_r_0010_1=np.max(N_r_0010_1)
N_05_r_0010_1=np.percentile(N_r_0010_1,95)

# R0010_segment_binaural_44100_2.wav
#L
N_l_0010_2, N_spec_l_0010_2, bark_axis_l_0010_2, time_axis_l_0010_2 = loudness_zwtv(sigL_0010_2, fs, field_type="free")
N_avg_l_0010_2=np.mean(N_l_0010_2)
N_max_l_0010_2=np.max(N_l_0010_2)
N_05_l_0010_2=np.percentile(N_l_0010_2,95)
#R
N_r_0010_2, N_spec_r_0010_2, bark_axis_r_0010_2, time_axis_r_0010_2 = loudness_zwtv(sigR_0010_2, fs, field_type="free")
N_avg_r_0010_2=np.mean(N_r_0010_2)
N_max_r_0010_2=np.max(N_r_0010_2)
N_05_r_0010_2=np.percentile(N_r_0010_2,95)

In [ ]:
N_avg_l=np.array([N_avg_l_0001_1,N_avg_l_0001_2
                  ,N_avg_l_0002_1, N_avg_l_0002_2,
                  N_avg_l_0003_1, N_avg_l_0003_2,
                  N_avg_l_0004_1, N_avg_l_0004_2,
                  N_avg_l_0005_1, N_avg_l_0005_2,
                  N_avg_l_0006_1, N_avg_l_0006_2,
                  N_avg_l_0007_1, N_avg_l_0007_2,
                  N_avg_l_0008_1, N_avg_l_0008_2,
                  N_avg_l_0009_1, N_avg_l_0009_2,
                  N_avg_l_0010_1, N_avg_l_0010_2,
                  ])

N_avg_r=np.array([N_avg_r_0001_1,N_avg_r_0001_2
                  ,N_avg_r_0002_1, N_avg_r_0002_2,
                  N_avg_r_0003_1, N_avg_r_0003_2,
                  N_avg_r_0004_1, N_avg_r_0004_2,
                  N_avg_r_0005_1, N_avg_r_0005_2,
                  N_avg_r_0006_1, N_avg_r_0006_2,
                  N_avg_r_0007_1, N_avg_r_0007_2,
                  N_avg_r_0008_1, N_avg_r_0008_2,
                  N_avg_r_0009_1, N_avg_r_0009_2,
                  N_avg_r_0010_1, N_avg_r_0010_2,
                  ])

N_max_l=np.array([ N_max_l_0001_1, N_max_l_0001_2,
                  N_max_l_0002_1, N_max_l_0002_2,
                  N_max_l_0003_1, N_max_l_0003_2,
                  N_max_l_0004_1, N_max_l_0004_2,
                  N_max_l_0005_1, N_max_l_0005_2,
                  N_max_l_0006_1, N_max_l_0006_2,
                  N_max_l_0007_1, N_max_l_0007_2,
                  N_max_l_0008_1, N_max_l_0008_2,
                  N_max_l_0009_1, N_max_l_0009_2,
                  N_max_l_0010_1, N_max_l_0010_2,
                ])
N_max_r=np.array([ N_max_r_0001_1, N_max_r_0001_2,
                  N_max_r_0002_1, N_max_r_0002_2,
                  N_max_r_0003_1, N_max_r_0003_2,
                  N_max_r_0004_1, N_max_r_0004_2,
                  N_max_r_0005_1, N_max_r_0005_2,
                  N_max_r_0006_1, N_max_r_0006_2,
                  N_max_r_0007_1, N_max_r_0007_2,
                  N_max_r_0008_1, N_max_r_0008_2,
                  N_max_r_0009_1, N_max_r_0009_2,
                  N_max_r_0010_1, N_max_r_0010_2,
                ])

N_05_l=np.array([
    N_05_l_0001_1, N_05_l_0001_2,
    N_05_l_0002_1, N_05_l_0002_2,
    N_05_l_0003_1, N_05_l_0003_2,
    N_05_l_0004_1, N_05_l_0004_2,
    N_05_l_0005_1, N_05_l_0005_2,
    N_05_l_0006_1, N_05_l_0006_2,
    N_05_l_0007_1, N_05_l_0007_2,
    N_05_l_0008_1, N_05_l_0008_2,
    N_05_l_0009_1, N_05_l_0009_2,
    N_05_l_0010_1, N_05_l_0010_2,
])

N_05_r=np.array([
    N_05_r_0001_1, N_05_r_0001_2,
    N_05_r_0002_1, N_05_r_0002_2,
    N_05_r_0003_1, N_05_r_0003_2,
    N_05_r_0004_1, N_05_r_0004_2,
    N_05_r_0005_1, N_05_r_0005_2,
    N_05_r_0006_1, N_05_r_0006_2,
    N_05_r_0007_1, N_05_r_0007_2,
    N_05_r_0008_1, N_05_r_0008_2,
    N_05_r_0009_1, N_05_r_0009_2,
    N_05_r_0010_1, N_05_r_0010_2,
])

N_avg=(N_avg_l+N_avg_r)/2
N_max=(N_max_l+N_max_r)/2
N_05=(N_05_l+N_05_r)/2

In [ ]:
# Average loudness
plt.figure()
plt.plot(N_avg, "o-", label="result")
plt.plot(ground_truth[:,1], "o-", label="ground truth")
#plt.plot(N_avg_r, "o-", label="result_R")
#plt.plot(N_avg_l, "o-", label="result_L")
plt.xlabel('Soundscapes')
plt.title("Average Loudness (N_avg_s)")
plt.legend()
plt.show()

In [ ]:
# Maximum loudness
plt.figure()
plt.plot(N_max, "o-", label="result")
plt.plot(ground_truth[:,2], "o-", label="ground truth")
#plt.plot(N_max_r, "o-", label="result_R")
#plt.plot(N_max_l, "o-", label="result_L")
plt.xlabel('Soundscapes')
plt.title("Max Loudness (N_max_s)")
plt.legend()
plt.show()

In [ ]:
# 05-Percentile loudness
plt.figure()
plt.plot(N_05, "o-", label="result")
plt.plot(ground_truth[:,3], "o-", label="ground truth")
#plt.plot(N_05_r, "o-", label="result_R")
#plt.plot(N_05_l, "o-", label="result_L")
plt.xlabel('Soundscapes')
plt.title("05-percentile Loudness (N_05_s)")
plt.legend()
plt.show()

# Comparing different non-augmented soundscapes calculation VS ground-truth MORE PARAMETRES- loudness

# Comparing different non-augmented soundscapes calculation VS ground-truth - sharpness

In [ ]:
# repeat with 

# R0001_segment_binaural_44100_1.wav
#L
S_l_0001_1, time_axis_l_0001_1 = sharpness_din_tv(sigL_0001_1, fs, field_type="free", skip=0.5)
S_avg_l_0001_1=np.mean(S_l_0001_1)
S_max_l_0001_1=np.max(S_l_0001_1)
S_05_l_0001_1=np.percentile(S_l_0001_1,95)
#R
S_r_0001_1, time_axis_r_0001_1 = sharpness_din_tv(sigR_0001_1, fs, field_type="free", skip=0.5)
S_avg_r_0001_1=np.mean(S_r_0001_1)
S_max_r_0001_1=np.max(S_r_0001_1)
S_05_r_0001_1=np.percentile(S_r_0001_1,95)

# R0001_segment_binaural_44100_2.wav
#L
S_l_0001_2, time_axis_l_0001_2 = sharpness_din_tv(sigL_0001_2, fs, field_type="free", skip=0.5)
S_avg_l_0001_2=np.mean(S_l_0001_2)
S_max_l_0001_2=np.max(S_l_0001_2)
S_05_l_0001_2=np.percentile(S_l_0001_2,95)
#R
S_r_0001_2, time_axis_r_0001_2 = sharpness_din_tv(sigR_0001_2, fs, field_type="free", skip=0.5)
S_avg_r_0001_2=np.mean(S_r_0001_2)
S_max_r_0001_2=np.max(S_r_0001_2)
S_05_r_0001_2=np.percentile(S_r_0001_2,95)

# R0002_segment_binaural_44100_1.wav
#L
S_l_0002_1, time_axis_l_0002_1 = sharpness_din_tv(sigL_0002_1, fs, field_type="free", skip=0.5)
S_avg_l_0002_1=np.mean(S_l_0002_1)
S_max_l_0002_1=np.max(S_l_0002_1)
S_05_l_0002_1=np.percentile(S_l_0002_1,95)
#R
S_r_0002_1, time_axis_r_0002_1 = sharpness_din_tv(sigR_0002_1, fs, field_type="free", skip=0.5)
S_avg_r_0002_1=np.mean(S_r_0002_1)
S_max_r_0002_1=np.max(S_r_0002_1)
S_05_r_0002_1=np.percentile(S_r_0002_1,95)

# R0002_segment_binaural_44100_2.wav
#L
S_l_0002_2, time_axis_l_0002_2 = sharpness_din_tv(sigL_0002_2, fs, field_type="free", skip=0.5)
S_avg_l_0002_2=np.mean(S_l_0002_2)
S_max_l_0002_2=np.max(S_l_0002_2)
S_05_l_0002_2=np.percentile(S_l_0002_2,95)
#R
S_r_0002_2, time_axis_r_0002_2 = sharpness_din_tv(sigR_0002_2, fs, field_type="free", skip=0.5)
S_avg_r_0002_2=np.mean(S_r_0002_2)
S_max_r_0002_2=np.max(S_r_0002_2)
S_05_r_0002_2=np.percentile(S_r_0002_2,95)

# R0003_segment_binaural_44100_1.wav
#L
S_l_0003_1,time_axis_l_0003_1 = sharpness_din_tv(sigL_0003_1, fs, field_type="free", skip=0.5)
S_avg_l_0003_1=np.mean(S_l_0003_1)
S_max_l_0003_1=np.max(S_l_0003_1)
S_05_l_0003_1=np.percentile(S_l_0003_1,95)
#R
S_r_0003_1,  time_axis_r_0003_1 = sharpness_din_tv(sigR_0003_1, fs, field_type="free", skip=0.5)
S_avg_r_0003_1=np.mean(S_r_0003_1)
S_max_r_0003_1=np.max(S_r_0003_1)
S_05_r_0003_1=np.percentile(S_r_0003_1,95)

# R0003_segment_binaural_44100_2.wav
#L
S_l_0003_2,  time_axis_l_0003_2 = sharpness_din_tv(sigL_0003_2, fs, field_type="free", skip=0.5)
S_avg_l_0003_2=np.mean(S_l_0003_2)
S_max_l_0003_2=np.max(S_l_0003_2)
S_05_l_0003_2=np.percentile(S_l_0003_2,95)
#R
S_r_0003_2, time_axis_r_0003_2 = sharpness_din_tv(sigR_0003_2, fs, field_type="free", skip=0.5)
S_avg_r_0003_2=np.mean(S_r_0003_2)
S_max_r_0003_2=np.max(S_r_0003_2)
S_05_r_0003_2=np.percentile(S_r_0003_2,95)

# R0004_segment_binaural_44100_1.wav
#L
S_l_0004_1, time_axis_l_0004_1 = sharpness_din_tv(sigL_0004_1, fs, field_type="free", skip=0.5)
S_avg_l_0004_1=np.mean(S_l_0004_1)
S_max_l_0004_1=np.max(S_l_0004_1)
S_05_l_0004_1=np.percentile(S_l_0004_1,95)
#R
S_r_0004_1, time_axis_r_0004_1 = sharpness_din_tv(sigR_0004_1, fs, field_type="free", skip=0.5)
S_avg_r_0004_1=np.mean(S_r_0004_1)
S_max_r_0004_1=np.max(S_r_0004_1)
S_05_r_0004_1=np.percentile(S_r_0004_1,95)

# R0004_segment_binaural_44100_2.wav
#L
S_l_0004_2,  time_axis_l_0004_2 = sharpness_din_tv(sigL_0004_2, fs, field_type="free", skip=0.5)
S_avg_l_0004_2=np.mean(S_l_0004_2)
S_max_l_0004_2=np.max(S_l_0004_2)
S_05_l_0004_2=np.percentile(S_l_0004_2,95)
#R
S_r_0004_2,  time_axis_r_0004_2 = sharpness_din_tv(sigR_0004_2, fs, field_type="free", skip=0.5)
S_avg_r_0004_2=np.mean(S_r_0004_2)
S_max_r_0004_2=np.max(S_r_0004_2)
S_05_r_0004_2=np.percentile(S_r_0004_2,95)

# R0005_segment_binaural_44100_1.wav
#L
S_l_0005_1, time_axis_l_0005_1 = sharpness_din_tv(sigL_0005_1, fs, field_type="free", skip=0.5)
S_avg_l_0005_1=np.mean(S_l_0005_1)
S_max_l_0005_1=np.max(S_l_0005_1)
S_05_l_0005_1=np.percentile(S_l_0005_1,95)
#R
S_r_0005_1,  time_axis_r_0005_1 = sharpness_din_tv(sigR_0005_1, fs, field_type="free", skip=0.5)
S_avg_r_0005_1=np.mean(S_r_0005_1)
S_max_r_0005_1=np.max(S_r_0005_1)
S_05_r_0005_1=np.percentile(S_r_0005_1,95)

# R0005_segment_binaural_44100_2.wav
#L
S_l_0005_2,  time_axis_l_0005_2 = sharpness_din_tv(sigL_0005_2, fs, field_type="free", skip=0.5)
S_avg_l_0005_2=np.mean(S_l_0005_2)
S_max_l_0005_2=np.max(S_l_0005_2)
S_05_l_0005_2=np.percentile(S_l_0005_2,95)
#R
S_r_0005_2, time_axis_r_0005_2 = sharpness_din_tv(sigR_0005_2, fs, field_type="free", skip=0.5)
S_avg_r_0005_2=np.mean(S_r_0005_2)
S_max_r_0005_2=np.max(S_r_0005_2)
S_05_r_0005_2=np.percentile(S_r_0005_2,95)

# R0006_segment_binaural_44100_1.wav
#L
S_l_0006_1,  time_axis_l_0006_1 = sharpness_din_tv(sigL_0006_1, fs, field_type="free", skip=0.5)
S_avg_l_0006_1=np.mean(S_l_0006_1)
S_max_l_0006_1=np.max(S_l_0006_1)
S_05_l_0006_1=np.percentile(S_l_0006_1,95)
#R
S_r_0006_1,  time_axis_r_0006_1 = sharpness_din_tv(sigR_0006_1, fs, field_type="free", skip=0.5)
S_avg_r_0006_1=np.mean(S_r_0006_1)
S_max_r_0006_1=np.max(S_r_0006_1)
S_05_r_0006_1=np.percentile(S_r_0006_1,95)

# R0006_segment_binaural_44100_2.wav
#L
S_l_0006_2,  time_axis_l_0006_2 = sharpness_din_tv(sigL_0006_2, fs, field_type="free", skip=0.5)
S_avg_l_0006_2=np.mean(S_l_0006_2)
S_max_l_0006_2=np.max(S_l_0006_2)
S_05_l_0006_2=np.percentile(S_l_0006_2,95)
#R
S_r_0006_2,  time_axis_r_0006_2 = sharpness_din_tv(sigR_0006_2, fs, field_type="free", skip=0.5)
S_avg_r_0006_2=np.mean(S_r_0006_2)
S_max_r_0006_2=np.max(S_r_0006_2)
S_05_r_0006_2=np.percentile(S_r_0006_2,95)

# R0007_segment_binaural_44100_1.wav
#L
S_l_0007_1, time_axis_l_0007_1 = sharpness_din_tv(sigL_0007_1, fs, field_type="free", skip=0.5)
S_avg_l_0007_1=np.mean(S_l_0007_1)
S_max_l_0007_1=np.max(S_l_0007_1)
S_05_l_0007_1=np.percentile(S_l_0007_1,95)
#R
S_r_0007_1, time_axis_r_0007_1 = sharpness_din_tv(sigR_0007_1, fs, field_type="free", skip=0.5)
S_avg_r_0007_1=np.mean(S_r_0007_1)
S_max_r_0007_1=np.max(S_r_0007_1)
S_05_r_0007_1=np.percentile(S_r_0007_1,95)

# R0007_segment_binaural_44100_2.wav
#L
S_l_0007_2,  time_axis_l_0007_2 = sharpness_din_tv(sigL_0007_2, fs, field_type="free", skip=0.5)
S_avg_l_0007_2=np.mean(S_l_0007_2)
S_max_l_0007_2=np.max(S_l_0007_2)
S_05_l_0007_2=np.percentile(S_l_0007_2,95)
#R
S_r_0007_2,  time_axis_r_0007_2 = sharpness_din_tv(sigR_0007_2, fs, field_type="free", skip=0.5)
S_avg_r_0007_2=np.mean(S_r_0007_2)
S_max_r_0007_2=np.max(S_r_0007_2)
S_05_r_0007_2=np.percentile(S_r_0007_2,95)

# R0008_segment_binaural_44100_1.wav
#L
S_l_0008_1, time_axis_l_0008_1 = sharpness_din_tv(sigL_0008_1, fs, field_type="free", skip=0.5)
S_avg_l_0008_1=np.mean(S_l_0008_1)
S_max_l_0008_1=np.max(S_l_0008_1)
S_05_l_0008_1=np.percentile(S_l_0008_1,95)
#R
S_r_0008_1,  time_axis_r_0008_1 = sharpness_din_tv(sigR_0008_1, fs, field_type="free", skip=0.5)
S_avg_r_0008_1=np.mean(S_r_0008_1)
S_max_r_0008_1=np.max(S_r_0008_1)
S_05_r_0008_1=np.percentile(S_r_0008_1,95)

# R0008_segment_binaural_44100_2.wav
#L
S_l_0008_2,  time_axis_l_0008_2 = sharpness_din_tv(sigL_0008_2, fs, field_type="free", skip=0.5)
S_avg_l_0008_2=np.mean(S_l_0008_2)
S_max_l_0008_2=np.max(S_l_0008_2)
S_05_l_0008_2=np.percentile(S_l_0008_2,95)
#R
S_r_0008_2, time_axis_r_0008_2 = sharpness_din_tv(sigR_0008_2, fs, field_type="free", skip=0.5)
S_avg_r_0008_2=np.mean(S_r_0008_2)
S_max_r_0008_2=np.max(S_r_0008_2)
S_05_r_0008_2=np.percentile(S_r_0008_2,95)

# R0009_segment_binaural_44100_1.wav
#L
S_l_0009_1,  time_axis_l_0009_1 = sharpness_din_tv(sigL_0009_1, fs, field_type="free", skip=0.5)
S_avg_l_0009_1=np.mean(S_l_0009_1)
S_max_l_0009_1=np.max(S_l_0009_1)
S_05_l_0009_1=np.percentile(S_l_0009_1,95)
#R
S_r_0009_1,time_axis_r_0009_1 = sharpness_din_tv(sigR_0009_1, fs, field_type="free", skip=0.5)
S_avg_r_0009_1=np.mean(S_r_0009_1)
S_max_r_0009_1=np.max(S_r_0009_1)
S_05_r_0009_1=np.percentile(S_r_0009_1,95)

# R0009_segment_binaural_44100_2.wav
#L
S_l_0009_2, time_axis_l_0009_2 = sharpness_din_tv(sigL_0009_2, fs, field_type="free", skip=0.5)
S_avg_l_0009_2=np.mean(S_l_0009_2)
S_max_l_0009_2=np.max(S_l_0009_2)
S_05_l_0009_2=np.percentile(S_l_0009_2,95)
#R
S_r_0009_2,  time_axis_r_0009_2 = sharpness_din_tv(sigR_0009_2, fs, field_type="free", skip=0.5)
S_avg_r_0009_2=np.mean(S_r_0009_2)
S_max_r_0009_2=np.max(S_r_0009_2)
S_05_r_0009_2=np.percentile(S_r_0009_2,95)

# R00010_segment_binaural_44100_1.wav
#L
S_l_0010_1,  time_axis_l_0010_1 = sharpness_din_tv(sigL_0010_1, fs, field_type="free", skip=0.5)
S_avg_l_0010_1=np.mean(S_l_0010_1)
S_max_l_0010_1=np.max(S_l_0010_1)
S_05_l_0010_1=np.percentile(S_l_0010_1,95)
#R
S_r_0010_1,  time_axis_r_0010_1 = sharpness_din_tv(sigR_0010_1, fs, field_type="free", skip=0.5)
S_avg_r_0010_1=np.mean(S_r_0010_1)
S_max_r_0010_1=np.max(S_r_0010_1)
S_05_r_0010_1=np.percentile(S_r_0010_1,95)

# R0010_segment_binaural_44100_2.wav
#L
S_l_0010_2,  time_axis_l_0010_2 = sharpness_din_tv(sigL_0010_2, fs, field_type="free", skip=0.5)
S_avg_l_0010_2=np.mean(S_l_0010_2)
S_max_l_0010_2=np.max(S_l_0010_2)
S_05_l_0010_2=np.percentile(S_l_0010_2,95)
#R
S_r_0010_2, time_axis_r_0010_2 = sharpness_din_tv(sigR_0010_2, fs, field_type="free", skip=0.5)
S_avg_r_0010_2=np.mean(S_r_0010_2)
S_max_r_0010_2=np.max(S_r_0010_2)
S_05_r_0010_2=np.percentile(S_r_0010_2,95)

In [ ]:
S_avg_l=np.array([S_avg_l_0001_1,S_avg_l_0001_2
                  ,S_avg_l_0002_1, S_avg_l_0002_2,
                  S_avg_l_0003_1, S_avg_l_0003_2,
                  S_avg_l_0004_1, S_avg_l_0004_2,
                  S_avg_l_0005_1, S_avg_l_0005_2,
                  S_avg_l_0006_1, S_avg_l_0006_2,
                  S_avg_l_0007_1, S_avg_l_0007_2,
                  S_avg_l_0008_1, S_avg_l_0008_2,
                  S_avg_l_0009_1, S_avg_l_0009_2,
                  S_avg_l_0010_1, S_avg_l_0010_2,
                  ])

S_avg_r=np.array([S_avg_r_0001_1,S_avg_r_0001_2
                  ,S_avg_r_0002_1, S_avg_r_0002_2,
                  S_avg_r_0003_1, S_avg_r_0003_2,
                  S_avg_r_0004_1, S_avg_r_0004_2,
                  S_avg_r_0005_1, S_avg_r_0005_2,
                  S_avg_r_0006_1, S_avg_r_0006_2,
                  S_avg_r_0007_1, S_avg_r_0007_2,
                  S_avg_r_0008_1, S_avg_r_0008_2,
                  S_avg_r_0009_1, S_avg_r_0009_2,
                  S_avg_r_0010_1, S_avg_r_0010_2,
                  ])

S_max_l=np.array([ S_max_l_0001_1, S_max_l_0001_2,
                  S_max_l_0002_1, S_max_l_0002_2,
                  S_max_l_0003_1, S_max_l_0003_2,
                  S_max_l_0004_1, S_max_l_0004_2,
                  S_max_l_0005_1, S_max_l_0005_2,
                  S_max_l_0006_1, S_max_l_0006_2,
                  S_max_l_0007_1, S_max_l_0007_2,
                  S_max_l_0008_1, S_max_l_0008_2,
                  S_max_l_0009_1, S_max_l_0009_2,
                  S_max_l_0010_1, S_max_l_0010_2,
                ])
S_max_r=np.array([ S_max_r_0001_1, S_max_r_0001_2,
                  S_max_r_0002_1, S_max_r_0002_2,
                  S_max_r_0003_1, S_max_r_0003_2,
                  S_max_r_0004_1, S_max_r_0004_2,
                  S_max_r_0005_1, S_max_r_0005_2,
                  S_max_r_0006_1, S_max_r_0006_2,
                  S_max_r_0007_1, S_max_r_0007_2,
                  S_max_r_0008_1, S_max_r_0008_2,
                  S_max_r_0009_1, S_max_r_0009_2,
                  S_max_r_0010_1, S_max_r_0010_2,
                ])

S_05_l=np.array([
    S_05_l_0001_1, S_05_l_0001_2,
    S_05_l_0002_1, S_05_l_0002_2,
    S_05_l_0003_1, S_05_l_0003_2,
    S_05_l_0004_1, S_05_l_0004_2,
    S_05_l_0005_1, S_05_l_0005_2,
    S_05_l_0006_1, S_05_l_0006_2,
    S_05_l_0007_1, S_05_l_0007_2,
    S_05_l_0008_1, S_05_l_0008_2,
    S_05_l_0009_1, S_05_l_0009_2,
    S_05_l_0010_1, S_05_l_0010_2,
])

S_05_r=np.array([
    S_05_r_0001_1, S_05_r_0001_2,
    S_05_r_0002_1, S_05_r_0002_2,
    S_05_r_0003_1, S_05_r_0003_2,
    S_05_r_0004_1, S_05_r_0004_2,
    S_05_r_0005_1, S_05_r_0005_2,
    S_05_r_0006_1, S_05_r_0006_2,
    S_05_r_0007_1, S_05_r_0007_2,
    S_05_r_0008_1, S_05_r_0008_2,
    S_05_r_0009_1, S_05_r_0009_2,
    S_05_r_0010_1, S_05_r_0010_2,
])

S_avg=(S_avg_l+S_avg_r)/2
S_max=(S_max_l+S_max_r)/2
S_05=(S_05_l+S_05_r)/2

In [ ]:
# Average sharpness
plt.figure()
plt.plot(S_avg, "o-", label="result")
plt.plot(ground_truth[:,14], "o-", label="ground truth")
#plt.plot(N_avg_r, "o-", label="result_R")
#plt.plot(N_avg_l, "o-", label="result_L")
plt.xlabel('Soundscapes')
plt.ylim(0,5)
plt.title("Average Sharpness (S_avg_s)")
plt.legend()
plt.show()

# Max sharpness
plt.figure()
plt.plot(S_max, "o-", label="result")
plt.plot(ground_truth[:,15], "o-", label="ground truth")
#plt.plot(N_avg_r, "o-", label="result_R")
#plt.plot(N_avg_l, "o-", label="result_L")
plt.xlabel('Soundscapes')
plt.ylim(0,5)
plt.title("Max Sharpness (S_max_s)")
plt.legend()
plt.show()

# 5% percentile sharpness
plt.figure()
plt.plot(S_05, "o-", label="result")
plt.plot(ground_truth[:,16], "o-", label="ground truth")
#plt.plot(N_avg_r, "o-", label="result_R")
#plt.plot(N_avg_l, "o-", label="result_L")
plt.ylim(0,5)
plt.xlabel('Soundscapes')
plt.title("5-percentile Sharpness (S_05_s)")
plt.legend()
plt.show()

In [ ]:
# Apparently, gain=4.6 gives the best results, lets check with more features values

N_avg_true=ground_truth[0,1]
N_max_true=ground_truth[0,2]
N_05_true=ground_truth[0,3]
N_10_true=ground_truth[0,4]
N_20_true=ground_truth[0,5]
N_30_true=ground_truth[0,6]
N_40_true=ground_truth[0,7]
N_50_true=ground_truth[0,8]
N_60_true=ground_truth[0,9]
N_70_true=ground_truth[0,10]
N_80_true=ground_truth[0,11]
N_90_true=ground_truth[0,12]
N_95_true=ground_truth[0,13]

# LOAD R0001_segment_binaural_44100_1.wav
path = "../data/soundscapes/R0001_segment_binaural_44100_1.wav"
sigL_test, fs = load(path, wav_calib=4.6, ch=0) #L
sigR_test, fs = load(path, wav_calib=4.6, ch=1) #R

# Calculate loudness
#L
N_l_test, N_spec_l_test, bark_axis_l_test, time_axis_l_test = loudness_zwtv(sigL_test, fs, field_type="free")
N_avg_l_test=np.mean(N_l_test)
N_max_l_test=np.max(N_l_test)
N_05_l_test=np.percentile(N_l_test,95)
N_10_l_test=np.percentile(N_l_test,90)
N_20_l_test=np.percentile(N_l_test,80)
N_30_l_test=np.percentile(N_l_test,70)
N_40_l_test=np.percentile(N_l_test,60)
N_50_l_test=np.percentile(N_l_test,50)
N_60_l_test=np.percentile(N_l_test,40)
N_70_l_test=np.percentile(N_l_test,30)
N_80_l_test=np.percentile(N_l_test,20)
N_90_l_test=np.percentile(N_l_test,10)
N_95_l_test=np.percentile(N_l_test,5)
print("loudness L calculated")
#R
N_r_test, N_spec_r_test, bark_axis_r_test, time_axis_r_test = loudness_zwtv(sigR_test, fs, field_type="free")
N_avg_r_test=np.mean(N_r_test)
N_max_r_test=np.max(N_r_test)
N_05_r_test=np.percentile(N_r_test,95)
N_10_r_test=np.percentile(N_r_test,90)
N_20_r_test=np.percentile(N_r_test,80)
N_30_r_test=np.percentile(N_r_test,70)
N_40_r_test=np.percentile(N_r_test,60)
N_50_r_test=np.percentile(N_r_test,50)
N_60_r_test=np.percentile(N_r_test,40)
N_70_r_test=np.percentile(N_r_test,30)
N_80_r_test=np.percentile(N_r_test,20)
N_90_r_test=np.percentile(N_r_test,10)
N_95_r_test=np.percentile(N_r_test,5)
print("loudness R calculated")

N_avg_test=(N_avg_r_test+ N_avg_l_test)/2
N_max_test=(N_max_r_test+ N_max_l_test)/2
N_05_test=(N_05_r_test+N_05_l_test)/2
N_10_test=(N_10_r_test+N_10_l_test)/2
N_20_test=(N_20_r_test+N_20_l_test)/2
N_30_test=(N_30_r_test+N_30_l_test)/2
N_40_test=(N_40_r_test+N_40_l_test)/2
N_50_test=(N_50_r_test+N_50_l_test)/2
N_60_test=(N_60_r_test+N_60_l_test)/2
N_70_test=(N_70_r_test+N_70_l_test)/2
N_80_test=(N_80_r_test+N_80_l_test)/2
N_90_test=(N_90_r_test+N_90_l_test)/2
N_95_test=(N_95_r_test+N_95_l_test)/2

print("Navg, ground truth ",N_avg_true ," and result ",N_avg_test )
print("Nmax, ground truth ",N_max_true ," and result ", N_max_test)
print("N05, ground truth ",N_05_true ," and result ",N_05_test )
print("N10, ground truth ",N_10_true ," and result ",N_10_test )
print("N20, ground truth ",N_20_true ," and result ",N_20_test )
print("N30, ground truth ",N_30_true ," and result ",N_30_test )
print("N40, ground truth ",N_40_true ," and result ",N_40_test )
print("N50, ground truth ",N_50_true ," and result ",N_50_test )
print("N60, ground truth ",N_60_true ," and result ",N_60_test )
print("N70, ground truth ",N_70_true ," and result ",N_70_test )
print("N80, ground truth ",N_80_true ," and result ",N_80_test )
print("N90, ground truth ",N_90_true ," and result ",N_90_test )
print("N95, ground truth ",N_95_true ," and result ",N_95_test )

# Comparing different non-augmented soundscapes calculation VS ground-truth - roughness

In [ ]:
R_avg_true=0.0234
R_max_true=0.0684
R_05_true=0.034

# LOAD R0001_segment_binaural_44100_1.wav
path = "../data/soundscapes/R0001_segment_binaural_44100_1.wav"
sigL_test, fs = load(path, wav_calib=4.6, ch=0) #L
sigR_test, fs = load(path, wav_calib=4.6, ch=1) #R

# Calculate sharpness
#L
R_l_test, R_spec_l_test, _, _= roughness_dw(sigL_test, fs, 0)
R_avg_l_test=np.mean(R_l_test)
R_max_l_test=np.max(R_l_test)
R_05_l_test=np.percentile(R_l_test,95)
#R
R_r_test,R_spec_r_test,_,_ = roughness_dw(sigR_test, fs, 0)
R_avg_r_test=np.mean(R_r_test)
R_max_r_test=np.max(R_r_test)
R_05_r_test=np.percentile(R_r_test,95)

R_avg_test=(R_avg_r_test+ R_avg_l_test)/2
R_max_test=(R_max_r_test+ R_max_l_test)/2
R_05_test=(R_05_r_test+R_05_l_test)/2

print("R_avg, ground truth ",R_avg_true ," and result ",R_avg_test )
print("Rmax, ground truth ",R_max_true ," and result ", R_max_test)
print("R05, ground truth ",R_05_true ," and result ",R_05_test )

# NOT THE CORRECT FORMULA


# Comparing different non-augmented soundscapes calculation VS ground-truth - tonality

In [ ]:
# Local functions imports
from mosqito.utils.time_segmentation import time_segmentation
from mosqito.sound_level_meter.spectrum import spectrum
from mosqito.sq_metrics.tonality.tone_to_noise_ecma._critical_band import _critical_band
from mosqito.sq_metrics.tonality.tone_to_noise_ecma._screening_for_tones import _screening_for_tones
from mosqito.sq_metrics.tonality.tone_to_noise_ecma._find_highest_tone import _find_highest_tone
from mosqito.sq_metrics.tonality.tone_to_noise_ecma._peak_level import _peak_level


def _tnr_main_calc(spectrum_db, freq_axis):

    #### Spectrum creation #######################################################


    if len(spectrum_db.shape) == 1:
        nseg = 1
        # Frequency axis of interest
        freq_index = np.where((freq_axis > 20) & (freq_axis < 20000))[0]
        freqs = freq_axis[freq_index]
        spec_db = spectrum_db[freq_index]
        
    elif (len(spectrum_db.shape) > 1) & (len(freq_axis.shape) > 1):
        nseg = spectrum_db.shape[1]
        freqs = [[]for i in range(nseg)]
        spec_db = [[]for i in range(nseg)]
        for i in range(nseg):
            # Frequency axis of interest
            freq_index_rows = np.where((freq_axis[:,i] > 20) & (freq_axis[:,i] < 20000))[0]
            freqs[i] = np.append(freqs[i],freq_axis[freq_index_rows,i])
            spec_db[i] = np.append(spec_db[i],spectrum_db[freq_index_rows,i])
        freqs = np.asarray(freqs)
        spec_db = np.asarray(spec_db)
    
    elif (len(spectrum_db.shape) > 1) & (len(freq_axis.shape) == 1):
        # Frequency axis of interest
        freq_index = np.where((freq_axis > 20) & (freq_axis < 20000))[0]
        # Initialization
        nfreqs = len(freq_index)    
        nseg = spectrum_db.shape[1]
        freqs = np.zeros((nseg,nfreqs))
        spec_db = np.zeros((nseg,nfreqs))
        for i in range(nseg):
            freqs[i,:] = freq_axis[freq_index]
            spec_db[i,:] = spectrum_db[freq_index,i]


    #### Screening to find the potential tonal components ########################

    peak_index = _screening_for_tones(freqs, spec_db, "smoothed", 20, 20000)

    # Initialization of the results lists
    if nseg == 1:
        TNR = []
        t_tnr = []
        tones_freqs = []
        prominence = []
    else:   
        TNR = [[]for i in range(nseg)]
        t_tnr = [[]for i in range(nseg)]
        tones_freqs = [[]for i in range(nseg)]
        prominence = [[]for i in range(nseg)]
    

    #### Evaluation of each candidate ############################################

    for i in range(nseg):
        
        tnr = np.array([], dtype=object)
        
        if nseg == 1:
            peaks = peak_index
            spec = spec_db
            fr = freqs
            frs = freq_axis
        elif nseg > 1:
            peaks = peak_index[i]
            spec = spec_db[i,:]
            fr = freqs[i,:]
            if len(freq_axis.shape)>1:
                frs = freq_axis[:,i]
            else:
                frs = freq_axis
        
        nb_tones = len(peaks)

        # Each candidate is studied and then deleted from the list until all have been treated
        while nb_tones > 0:
            ind = int(peaks[0])
            if len(peaks) > 1:
                ind_p, ind_s, peaks, nb_tones = _find_highest_tone(
                    fr, spec, peaks.astype(int), nb_tones, ind
                )
            else:
                ind_p = ind
                ind_s = None
    
            # multiple tones in a critical band
            if ind_s != None:
                fp = fr[ind_p]
                fs = fr[ind_s]
    
                # proximity criterion
                delta_f = 21 * 10 ** ((1.2 * (np.abs(np.log10(fp / 212))) ** 1.8))
                if np.abs(fs - fp) < delta_f:
    
                    # tone SPL
                    Lp = _peak_level(fr, spec, ind_p)
                    Ls = _peak_level(fr, spec, ind_s)
    
                    Lt = 10 * np.log10(((10 ** (Lp / 10) + 10 ** (Ls / 10))))
    
                    # total SPL in the critical band
                    f1, f2 = _critical_band(fp)
                    low_limit_idx = np.argmin(np.abs(fr - f1))
                    high_limit_idx = np.argmin(np.abs(fr - f2))
    
                    spec_sum = sum(10 ** (spec[low_limit_idx:high_limit_idx] / 10))
                    Ltot = 10 * np.log10(spec_sum)
    
                    # suppression of the second highest tone from the list of tones
                    sup = np.where(peaks == ind_s)[0]
                    peaks = np.delete(peaks, sup)
                    nb_tones -= 1
    
                    delta_ft = 2 * (frs[1] - frs[0])
    
                else:
                    # the two highest tones are not close enough to be considered as one
                    # tone SPL
                    Lt = spec[ind_p]
    
                    # total SPL in the critical band
                    f1, f2 = _critical_band(fr[ind_p])
                    low_limit_idx = np.argmin(np.abs(fr - f1))
                    high_limit_idx = np.argmin(np.abs(fr - f2))
    
                    spec_sum = sum(10 ** (spec[low_limit_idx:high_limit_idx] / 10))
                    Ltot = 10 * np.log10(spec_sum)
    
                    delta_ft = fr[1] - fr[0]
    
            # single tone in a critical band
            else:
                # tone SPL
                Lt = _peak_level(fr, spec, ind_p)
    
                # total SPL in the critical band
                f1, f2 = _critical_band(fr[ind_p])
                low_limit_idx = np.argmin(np.abs(fr - f1))
                high_limit_idx = np.argmin(np.abs(fr - f2))
    
                spec_sum = sum(10 ** (spec[low_limit_idx:high_limit_idx] / 10))
                Ltot = 10 * np.log10(spec_sum)
    
                delta_ft = fr[1] - fr[0]
    
            # SPL of the masking noise
            delta_fc = f2 - f1
            delta_ftot = frs[high_limit_idx] - frs[low_limit_idx]
            Ln = 10 * np.log10(10 ** (Ltot / 10) - 10 ** (Lt / 10)) + 10 * np.log10(
                delta_fc / (delta_ftot - delta_ft)
            )
    
            # Tone-to-noise ratio
            f = fr[ind_p]
            delta_t = Lt - Ln
            if delta_t > 0:
                if nseg > 1:
                    tones_freqs[i] = np.append(tones_freqs[i], f)
                elif nseg == 1:
                    tones_freqs = np.append(tones_freqs, f)
                tnr = np.append(tnr, delta_t)
    
                # Prominence criteria
                if f >= 20 and f < 1000:
                    if delta_t >= 8 + 8.33 * np.log10(1000 / f):
                        if nseg > 1:
                            prominence[i].append(True)                        
                        elif nseg == 1:
                            prominence.append(True)
                    else:
                        if nseg > 1:
                            prominence[i].append(False)
                        
                        elif nseg == 1:
                            prominence.append(False)
                elif f >= 1000 and f <= 20000:
                    if delta_t >= 8:
                        if nseg > 1:
                            prominence[i].append(True)
                        
                        elif nseg == 1:
                            prominence.append(True)
                    else:
                        if nseg > 1:
                            prominence[i].append(False)
                        
                        elif nseg == 1:
                            prominence.append(False)
    
            # suppression from the list of tones
            sup = np.where(peaks == ind_p)[0]
            peaks = np.delete(peaks, sup)
            nb_tones -= 1
    

        if nseg > 1:
            if sum(np.power(10, (tnr[prominence[i]] / 10))) != 0:
                t_tnr[i] = 10 * np.log10(sum(np.power(10, (tnr[prominence[i]] / 10))))
            else:
                t_tnr[i] =  0
            TNR[i] = np.append(TNR[i], tnr)               
                    
                    
        elif nseg == 1:
            if sum(np.power(10, (tnr[prominence] / 10))) != 0:
                t_tnr = np.append(t_tnr,10 * np.log10(sum(np.power(10, (tnr[prominence] / 10)))))
            else:
                t_tnr =  0
            TNR = np.append(TNR, tnr)
        
    tones_freqs = np.asarray(tones_freqs, dtype=object)
    prominence = np.asarray(prominence, dtype=object)

    
    return tones_freqs, TNR , prominence, t_tnr


def tnr_ecma_tv(signal, fs, prominence=False, overlap=0):
    
    if len(signal.shape) == 1:
      
        # Number of points within each frame according to the time resolution of 500ms
        nperseg = int(1 * fs)
        # Overlappinf segment length
        noverlap = int(overlap * nperseg)               
        # Time segmentation of the signal
        sig, time = time_segmentation(signal, fs, nperseg=nperseg, noverlap=noverlap, is_ecma=True)
        # Number of segments
        nseg = sig.shape[1] 
        # Spectrum computation
        spectrum_db, freq_axis = spectrum(sig, fs, db=True)
      
    else:
        nseg = signal.shape[1]
        time = np.linspace(0, signal.shape[0]/fs, num=nseg)
        
        # Compute spectrum
        spectrum_db, freq_axis = spectrum(sig, fs, db=True)
            
            
    # compute tnr values
    tones_freqs, tnr_, prom, t_tnr = _tnr_main_calc(spectrum_db, freq_axis)
 
            
    # Retore the results in a time vs frequency array
    freqs = np.logspace(np.log10(20), np.log10(20000), num=1000)
    tnr = np.zeros((len(freqs), nseg))
    print("shapeee ", tnr.shape)
    #tnr.fill(np.nan)
    promi = np.empty((len(freqs), nseg), dtype=bool)
    promi.fill(False)
    
    for t in range(nseg):
        for f in range(len(tones_freqs[t])):
            ind = np.argmin(np.abs(freqs - tones_freqs[t][f]))
            print(ind)
            if prominence == False:
                tnr[ind, t] = tnr_[t][f]
                promi[ind, t] = prom[t][f]
            if prominence == True:
                if prom[t][f] == True:
                    tnr[ind, t] = tnr_[t][f]
                    promi[ind, t] = prom[t][f]
    t_tnr = np.ravel(t_tnr)

    return t_tnr, tnr, promi, freqs, time     




# lets try with tonality

T_avg_true=0.0592
T_max_true=1.04
T_05_true=0.29

# LOAD R0001_segment_binaural_44100_1.wav
path = "../data/soundscapes/R0001_segment_binaural_44100_1.wav"
sigL_test, fs = load(path, wav_calib=4.6, ch=0) #L
sigR_test, fs = load(path, wav_calib=4.6, ch=1) #R

# Calculate sharpness
#L
_, T_l_test, _, _,_= tnr_ecma_tv(sigL_test, fs,prominence=False, overlap=0)
print((np.sum(T_l_test, axis=1)).shape)
T_avg_l_test=np.mean(np.sum(T_l_test, axis=0))
T_max_l_test=np.max(T_l_test)
T_05_l_test=np.percentile(T_l_test,95)
#R
_, T_r_test, _, _,_= tnr_ecma_tv(sigR_test, fs,prominence=False, overlap=0)

T_avg_r_test=np.nanmean(T_r_test)
T_max_r_test=np.nanmax(T_r_test)
T_05_r_test=np.nanpercentile(T_r_test,95)

T_avg_test=(T_avg_r_test+ T_avg_l_test)/2
T_max_test=(T_max_r_test+ T_max_l_test)/2
T_05_test=(T_05_r_test+T_05_l_test)/2

print("T_avg, ground truth ",T_avg_true ," and result ",T_avg_test )
print("Tmax, ground truth ",T_max_true ," and result ", T_max_test)
print("T05, ground truth ",T_05_true ," and result ",T_05_test )

# Comparing different non-augmented soundscapes calculation VS ground-truth - LAeq

In [ ]:
from numpy import pi, convolve
from scipy.signal.filter_design import bilinear
from scipy.signal import lfilter

def A_weighting(Fs):
    """Design of an A-weighting filter.
    
    B, A = A_weighting(Fs) designs a digital A-weighting filter for 
    sampling frequency Fs. Usage: y = lfilter(B, A, x).
    Warning: Fs should normally be higher than 20 kHz. For example, 
    Fs = 48000 yields a class 1-compliant filter.
    
    Originally a MATLAB script. Also included ASPEC, CDSGN, CSPEC.
    
    Author: Christophe Couvreur, Faculte Polytechnique de Mons (Belgium)
            couvreur@thor.fpms.ac.be
    Last modification: Aug. 20, 1997, 10:00am.
    
    http://www.mathworks.com/matlabcentral/fileexchange/69
    http://replaygain.hydrogenaudio.org/mfiles/adsgn.m
    Translated from adsgn.m to PyLab 2009-07-14 endolith@gmail.com
    
    References: 
       [1] IEC/CD 1672: Electroacoustics-Sound Level Meters, Nov. 1996.
    
    """
    # Definition of analog A-weighting filter according to IEC/CD 1672.
    f1 = 20.598997
    f2 = 107.65265
    f3 = 737.86223
    f4 = 12194.217
    A1000 = 1.9997
    
    NUMs = [(2 * pi * f4)**2 * (10**(A1000/20)), 0, 0, 0, 0]
    DENs = convolve([1, +4 * pi * f4, (2 * pi * f4)**2], 
                    [1, +4 * pi * f1, (2 * pi * f1)**2], mode='full')
    DENs = convolve(convolve(DENs, [1, 2 * pi * f3], mode='full'), 
                                   [1, 2 * pi * f2], mode='full')
    
    # Use the bilinear transformation to get the digital filter.
    # (Octave, MATLAB, and PyLab disagree about Fs vs 1/Fs)
    return bilinear(NUMs, DENs, Fs)


def C_weighting(fs):
    f1 = 20.598997 
    f4 = 12194.217
    C1000 = 0.0619

    NUMs = [(2*pi*f4)**2*(10**(C1000/20.0)),0,0]
    DENs = np.polymul([1,4*pi*f4,(2*pi*f4)**2.0],[1,4*pi*f1,(2*pi*f1)**2]) 
    DENs = convolve([1, +4 * pi * f4, (2 * pi * f4)**2], 
                    [1, +4 * pi * f1, (2 * pi * f1)**2], mode='full')

    #Use the bilinear transformation to get the digital filter. 
    return bilinear(NUMs,DENs,fs)


"https://gist.github.com/endolith/148112/cbf8fea907ed3998d8469b776f7245adae862282"


In [ ]:

# LOAD R0001_segment_binaural_44100_1.wav
path = "../data/soundscapes/R0001_segment_binaural_44100_2.wav"
sigL, fs = load(path, wav_calib=4.6, ch=0) #L
sigR, fs = load(path, wav_calib=4.6, ch=1) #R


[B,A] = A_weighting(fs)
sigL_A = lfilter(B,A,sigL)
sigR_A = lfilter(B,A,sigR)

sigL_0001_1_A = lfilter(B,A,sigL_0001_1) #L
sigR_0001_1_A = lfilter(B,A,sigR_0001_1) #R

sigL_0001_2_A = lfilter(B,A,sigL_0001_2) #L
sigR_0001_2_A = lfilter(B,A,sigR_0001_2) #R

sigL_0002_1_A = lfilter(B,A,sigL_0002_1) #L
sigR_0002_1_A = lfilter(B,A,sigR_0002_1) #R

sigL_0002_2_A = lfilter(B,A,sigL_0002_2) #L
sigR_0002_2_A = lfilter(B,A,sigR_0002_2) #R

sigL_0003_1_A = lfilter(B,A,sigL_0003_1) #L
sigR_0003_1_A = lfilter(B,A,sigR_0003_1) #R

sigL_0003_2_A = lfilter(B,A,sigL_0003_2) #L
sigR_0003_2_A = lfilter(B,A,sigR_0003_2) #R

sigL_0004_1_A = lfilter(B,A,sigL_0004_1) #L
sigR_0004_1_A = lfilter(B,A,sigR_0004_1) #R

sigL_0004_2_A = lfilter(B,A,sigL_0004_2) #L
sigR_0004_2_A = lfilter(B,A,sigR_0004_2) #R

sigL_0005_1_A = lfilter(B,A,sigL_0005_1) #L
sigR_0005_1_A = lfilter(B,A,sigR_0005_1) #R

sigL_0005_2_A = lfilter(B,A,sigL_0005_2) #L
sigR_0005_2_A = lfilter(B,A,sigR_0005_2) #R

sigL_0006_1_A = lfilter(B,A,sigL_0006_1) #L
sigR_0006_1_A = lfilter(B,A,sigR_0006_1) #R

sigL_0006_2_A = lfilter(B,A,sigL_0006_2) #L
sigR_0006_2_A = lfilter(B,A,sigR_0006_2) #R

sigL_0007_1_A = lfilter(B,A,sigL_0007_1) #L
sigR_0007_1_A = lfilter(B,A,sigR_0007_1) #R

sigL_0007_2_A = lfilter(B,A,sigL_0007_2) #L
sigR_0007_2_A = lfilter(B,A,sigR_0007_2) #R

sigL_0008_1_A = lfilter(B,A,sigL_0008_1) #L
sigR_0008_1_A = lfilter(B,A,sigR_0008_1) #R

sigL_0008_2_A = lfilter(B,A,sigL_0008_2) #L
sigR_0008_2_A = lfilter(B,A,sigR_0008_2) #R

sigL_0009_1_A = lfilter(B,A,sigL_0009_1) #L
sigR_0009_1_A = lfilter(B,A,sigR_0009_1) #R

sigL_0009_2_A = lfilter(B,A,sigL_0009_2) #L
sigR_0009_2_A = lfilter(B,A,sigR_0009_2) #R

sigL_0010_1_A = lfilter(B,A,sigL_0010_1) #L
sigR_0010_1_A = lfilter(B,A,sigR_0010_1) #R

sigL_0010_2_A = lfilter(B,A,sigL_0010_2) #L
sigR_0010_2_A = lfilter(B,A,sigR_0010_2) #R


In [ ]:
LAeq_groundtruth = np.array([60.06, 59.76, 64.3, 65.63, 53.08, 50.41, 58.51, 58.98, 49.08, 52.16, 52.72, 58.87, 52.28, 51.27, 53.86, 53.66, 60.27, 60.47, 64.65, 63.83])
LAmax_groundtruth=np.array([67.63, 61.28, 69.91, 76.78, 61.19, 55.97, 61.61, 62.92, 56.75, 59.94, 56.88, 60.9, 60.79, 61.93, 54.78, 55.53, 61.63, 62.14, 71.58, 70.8])
LAmin_groundtruth=np.array([58.25, 58.21, 57.77, 56.93, 48.26, 48.08, 55.89, 56.79, 47.6, 48.47, 50.48, 53.29, 45.74, 45.92, 52.95, 52.03, 59.44, 59.11, 58.22, 58.91])
LA05_groundtruth=np.array([63.08, 60.58, 68.53, 74.19, 55.67, 53.95, 60.39, 61.15, 50.37, 54.95, 54.72, 60.27, 56.05, 56.22, 54.39, 54.27, 60.81, 61.02, 68.92, 67.08])


LAeqL_0001_1 = mean_dB(pressure2leq(sigL_0001_1_A, fs))
LAeqR_0001_1 = mean_dB(pressure2leq(sigR_0001_1_A, fs))
LAeqS_0001_1 =mean_dB([LAeqL_0001_1,LAeqR_0001_1])
LAmaxL_0001_1 =np.max(pressure2dBSPL(np.abs(sigL_0001_1_A)))
LAmaxR_0001_1 =np.max(pressure2dBSPL(np.abs(sigR_0001_1_A)))
LAmax_0001_1=mean_dB(LAmaxL_0001_1,LAmaxR_0001_1)
LAminL_0001_1 =np.nanmin(pressure2dBSPL(np.abs(sigL_0001_1_A)))
LAminR_0001_1 =np.nanmin(pressure2dBSPL(np.abs(sigL_0001_1_A)))
LAmin_0001_1=mean_dB(LAminL_0001_1,LAminR_0001_1)
LA05L_0001_1 =np.percentile(pressure2dBSPL(np.abs(sigL_0001_1_A)), 95)
LA05R_0001_1 =np.percentile(pressure2dBSPL(np.abs(sigL_0001_1_A)), 95)
LA05_0001_1=mean_dB(LA05L_0001_1,LA05R_0001_1)

print(pressure2dBSPL(sigL_0001_1_A), LAmaxL_0001_1, LAminL_0001_1, LA05L_0001_1)


LAeqL_0001_2 = mean_dB(pressure2leq(sigL_0001_2_A, fs))
LAeqR_0001_2 = mean_dB(pressure2leq(sigR_0001_2_A, fs))
LAeqS_0001_2 =mean_dB([LAeqL_0001_2,LAeqR_0001_2])
LAeqS_0001_2 =mean_dB([LAeqL_0001_1,LAeqR_0001_1])
LAmax_0001_2 =np.max(LAeqS_0001_2)
LAmin_0001_2 =np.min(LAeqS_0001_2)
LA05_0001_2 =np.percentile(LAeqS_0001_2, 95)

LAeqL_0002_1 = mean_dB(pressure2leq(sigL_0002_1_A, fs))
LAeqR_0002_1 = mean_dB(pressure2leq(sigR_0002_1_A, fs))
LAeqS_0002_1 = mean_dB([LAeqL_0002_1, LAeqR_0002_1])
LAmaxL_0002_1 = np.max(LAeqL_0002_1)
LAmaxR_0002_1 = np.max(LAeqR_0002_1)
LAmax_0002_1 = mean_dB([LAmaxL_0002_1, LAmaxR_0002_1])
LAminL_0002_1 = np.min(LAeqL_0002_1)
LAminR_0002_1 = np.min(LAeqR_0002_1)
LAmin_0002_1 = mean_dB([LAminL_0002_1, LAminR_0002_1])
LA05L_0002_1 = np.percentile(LAeqL_0002_1, 95)
LA05R_0002_1 = np.percentile(LAeqR_0002_1, 95)
LA05_0002_1 = mean_dB([LA05L_0002_1, LA05R_0002_1])

LAeqL_0002_2 = mean_dB(pressure2leq(sigL_0002_2_A, fs))
LAeqR_0002_2 = mean_dB(pressure2leq(sigR_0002_2_A, fs))
LAeqS_0002_2 =mean_dB([LAeqL_0002_2,LAeqR_0002_2])
LAmax_0002_2 =np.max(LAeqS_0002_2)
LAmin_0002_2 =np.min(LAeqS_0002_2)
LA05_0002_2 =np.percentile(LAeqS_0002_2, 95)

LAeqL_0003_1 = mean_dB(pressure2leq(sigL_0003_1_A, fs))
LAeqR_0003_1 = mean_dB(pressure2leq(sigR_0003_1_A, fs))
LAeqS_0003_1 =mean_dB([LAeqL_0003_1,LAeqR_0003_1])
LAmax_0003_1 =np.max(LAeqS_0003_1)
LAmin_0003_1 =np.min(LAeqS_0003_1)
LA05_0003_1 =np.percentile(LAeqS_0003_1, 95)

LAeqL_0003_2 = mean_dB(pressure2leq(sigL_0003_2_A, fs))
LAeqR_0003_2 = mean_dB(pressure2leq(sigR_0003_2_A, fs))
LAeqS_0003_2 =mean_dB([LAeqL_0003_2,LAeqR_0003_2])
LAmax_0003_2 =np.max(LAeqS_0003_2)
LAmin_0003_2 =np.min(LAeqS_0003_2)
LA05_0003_2 =np.percentile(LAeqS_0003_2, 95)

LAeqL_0004_1 = mean_dB(pressure2leq(sigL_0004_1_A, fs))
LAeqR_0004_1 = mean_dB(pressure2leq(sigR_0004_1_A, fs))
LAeqS_0004_1 =mean_dB([LAeqL_0004_1,LAeqR_0004_1])
LAmax_0004_1 =np.max(LAeqS_0004_1)
LAmin_0004_1 =np.min(LAeqS_0004_1)
LA05_0004_1 =np.percentile(LAeqS_0004_1, 95)

LAeqL_0004_2 = mean_dB(pressure2leq(sigL_0004_2_A, fs))
LAeqR_0004_2 = mean_dB(pressure2leq(sigR_0004_2_A, fs))
LAeqS_0004_2 =mean_dB([LAeqL_0004_2,LAeqR_0004_2])
LAmax_0004_2 =np.max(LAeqS_0004_2)
LAmin_0004_2 =np.min(LAeqS_0004_2)
LA05_0004_2 =np.percentile(LAeqS_0004_2, 95)

LAeqL_0005_1 = mean_dB(pressure2leq(sigL_0005_1_A, fs))
LAeqR_0005_1 = mean_dB(pressure2leq(sigR_0005_1_A, fs))
LAeqS_0005_1 =mean_dB([LAeqL_0005_1,LAeqR_0005_1])
LAmax_0005_1 =np.max(LAeqS_0005_1)
LAmin_0005_1 =np.min(LAeqS_0005_1)
LA05_0005_1 =np.percentile(LAeqS_0005_1, 95)

LAeqL_0005_2 = mean_dB(pressure2leq(sigL_0005_2_A, fs))
LAeqR_0005_2 = mean_dB(pressure2leq(sigR_0005_2_A, fs))
LAeqS_0005_2 =mean_dB([LAeqL_0005_2,LAeqR_0005_2])
LAmax_0005_2 =np.max(LAeqS_0005_2)
LAmin_0005_2 =np.min(LAeqS_0005_2)
LA05_0005_2 =np.percentile(LAeqS_0005_2, 95)

LAeqL_0006_1 = mean_dB(pressure2leq(sigL_0006_1_A, fs))
LAeqR_0006_1 = mean_dB(pressure2leq(sigR_0006_1_A, fs))
LAeqS_0006_1 =mean_dB([LAeqL_0006_1,LAeqR_0006_1])
LAmax_0006_1 =np.max(LAeqS_0006_1)
LAmin_0006_1 =np.min(LAeqS_0006_1)
LA05_0006_1 =np.percentile(LAeqS_0006_1, 95)

LAeqL_0006_2 = mean_dB(pressure2leq(sigL_0006_2_A, fs))
LAeqR_0006_2 = mean_dB(pressure2leq(sigR_0006_2_A, fs))
LAeqS_0006_2 =mean_dB([LAeqL_0006_2,LAeqR_0006_2])
LAmax_0006_2 =np.max(LAeqS_0006_2)
LAmin_0006_2 =np.min(LAeqS_0006_2)
LA05_0006_2 =np.percentile(LAeqS_0006_2, 95)

LAeqL_0007_1 = mean_dB(pressure2leq(sigL_0007_1_A, fs))
LAeqR_0007_1 = mean_dB(pressure2leq(sigR_0007_1_A, fs))
LAeqS_0007_1 =mean_dB([LAeqL_0007_1,LAeqR_0007_1])
LAmax_0007_1 =np.max(LAeqS_0007_1)
LAmin_0007_1 =np.min(LAeqS_0007_1)
LA05_0007_1 =np.percentile(LAeqS_0007_1, 95)

LAeqL_0007_2 = mean_dB(pressure2leq(sigL_0007_2_A, fs))
LAeqR_0007_2 = mean_dB(pressure2leq(sigR_0007_2_A, fs))
LAeqS_0007_2 =mean_dB([LAeqL_0007_2,LAeqR_0007_2])
LAmax_0007_2 =np.max(LAeqS_0007_2)
LAmin_0007_2 =np.min(LAeqS_0007_2)
LA05_0007_2 =np.percentile(LAeqS_0007_2, 95)

LAeqL_0008_1 = mean_dB(pressure2leq(sigL_0008_1_A, fs))
LAeqR_0008_1 = mean_dB(pressure2leq(sigR_0008_1_A, fs))
LAeqS_0008_1 =mean_dB([LAeqL_0008_1,LAeqR_0008_1])
LAmax_0008_1 =np.max(LAeqS_0008_1)
LAmin_0008_1 =np.min(LAeqS_0008_1)
LA05_0008_1 =np.percentile(LAeqS_0008_1, 95)

LAeqL_0008_2 = mean_dB(pressure2leq(sigL_0008_2_A, fs))
LAeqR_0008_2 = mean_dB(pressure2leq(sigR_0008_2_A, fs))
LAeqS_0008_2 =mean_dB([LAeqL_0008_2,LAeqR_0008_2])
LAmax_0008_2 =np.max(LAeqS_0008_2)
LAmin_0008_2 =np.min(LAeqS_0008_2)
LA05_0008_2 =np.percentile(LAeqS_0008_2, 95)

LAeqL_0009_1 = mean_dB(pressure2leq(sigL_0009_1_A, fs))
LAeqR_0009_1 = mean_dB(pressure2leq(sigR_0009_1_A, fs))
LAeqS_0009_1 =mean_dB([LAeqL_0009_1,LAeqR_0009_1])
LAmax_0009_1 =np.max(LAeqS_0009_1)
LAmin_0009_1 =np.min(LAeqS_0009_1)
LA05_0009_1 =np.percentile(LAeqS_0009_1, 95)

LAeqL_0009_2 = mean_dB(pressure2leq(sigL_0009_2_A, fs))
LAeqR_0009_2 = mean_dB(pressure2leq(sigR_0009_2_A, fs))
LAeqS_0009_2 =mean_dB([LAeqL_0009_2,LAeqR_0009_2])
LAmax_0009_2 =np.max(LAeqS_0009_2)
LAmin_0009_2 =np.min(LAeqS_0009_2)
LA05_0009_2 =np.percentile(LAeqS_0009_2, 95)

LAeqL_0010_1 = mean_dB(pressure2leq(sigL_0010_1_A, fs))
LAeqR_0010_1 = mean_dB(pressure2leq(sigR_0010_1_A, fs))
LAeqS_0010_1 =mean_dB([LAeqL_0010_1,LAeqR_0010_1])
LAmax_0010_1 =np.max(LAeqS_0010_1)
LAmin_0010_1 =np.min(LAeqS_0010_1)
LA05_0010_1 =np.percentile(LAeqS_0010_1, 95)

LAeqL_0010_2 = mean_dB(pressure2leq(sigL_0010_2_A, fs))
LAeqR_0010_2 = mean_dB(pressure2leq(sigR_0010_2_A, fs))
LAeqS_0010_2 =mean_dB([LAeqL_0010_2,LAeqR_0010_2])
LAmax_0010_2 =np.max(LAeqS_0010_2)
LAmin_0010_2 =np.min(LAeqS_0010_2)
LA05_0010_2 =np.percentile(LAeqS_0010_2, 95)

LAeqS_result=np.array([
    LAeqS_0001_1,
    LAeqS_0001_2,
    LAeqS_0002_1,
    LAeqS_0002_2,
    LAeqS_0003_1,
    LAeqS_0003_2,
    LAeqS_0004_1,
    LAeqS_0004_2,
    LAeqS_0005_1,
    LAeqS_0005_2,
    LAeqS_0006_1,
    LAeqS_0006_2,
    LAeqS_0007_1,
    LAeqS_0007_2,
    LAeqS_0008_1,
    LAeqS_0008_2,
    LAeqS_0009_1,
    LAeqS_0009_2,
    LAeqS_0010_1,
    LAeqS_0010_2,
])

LAmax_result=np.array([
    LAmax_0001_1,
    LAmax_0001_2,
    LAmax_0002_1,
    LAmax_0002_2,
    LAmax_0003_1,
    LAmax_0003_2,
    LAmax_0004_1,
    LAmax_0004_2,
    LAmax_0005_1,
    LAmax_0005_2,
    LAmax_0006_1,
    LAmax_0006_2,
    LAmax_0007_1,
    LAmax_0007_2,
    LAmax_0008_1,
    LAmax_0008_2,
    LAmax_0009_1,
    LAmax_0009_2,
    LAmax_0010_1,
    LAmax_0010_2,
])

LAmin_result=np.array([
    LAmin_0001_1,
    LAmin_0001_2,
    LAmin_0002_1,
    LAmin_0002_2,
    LAmin_0003_1,
    LAmin_0003_2,
    LAmin_0004_1,
    LAmin_0004_2,
    LAmin_0005_1,
    LAmin_0005_2,
    LAmin_0006_1,
    LAmin_0006_2,
    LAmin_0007_1,
    LAmin_0007_2,
    LAmin_0008_1,
    LAmin_0008_2,
    LAmin_0009_1,
    LAmin_0009_2,
    LAmin_0010_1,
    LAmin_0010_2,
])
LA05_result=np.array([
    LA05_0001_1,
    LA05_0001_2,
    LA05_0002_1,
    LA05_0002_2,
    LA05_0003_1,
    LA05_0003_2,
    LA05_0004_1,
    LA05_0004_2,
    LA05_0005_1,
    LA05_0005_2,
    LA05_0006_1,
    LA05_0006_2,
    LA05_0007_1,
    LA05_0007_2,
    LA05_0008_1,
    LA05_0008_2,
    LA05_0009_1,
    LA05_0009_2,
    LA05_0010_1,
    LA05_0010_2,
])

LAeq_error=LAeq_groundtruth-LAeqS_result
LAmax_error=LAmax_groundtruth-LAmax_result
LAmin_error=LAmin_groundtruth-LAmin_result
LA05_error=LA05_groundtruth-LA05_result


# Average LAeq
plt.figure()
plt.plot(LAeqS_result, "o-", label="result")
plt.plot(LAeq_groundtruth, "o-", label="ground truth")
plt.plot(LAeq_error, "o-", label="error")
#plt.plot(N_avg_r, "o-", label="result_R")
#plt.plot(N_avg_l, "o-", label="result_L")
plt.xlabel('Soundscapes')
plt.title("Average LAeq ")
plt.legend()
plt.show()

# Max LA
plt.figure()
plt.plot(LAmax_result, "o-", label="result")
plt.plot(LAmax_groundtruth, "o-", label="ground truth")
plt.plot(LAmax_error, "o-", label="error")
#plt.plot(N_avg_r, "o-", label="result_R")
#plt.plot(N_avg_l, "o-", label="result_L")
plt.xlabel('Soundscapes')
plt.title("Max LA ")
plt.legend()
plt.show()

# Min LA
plt.figure()
plt.plot(LAmin_result, "o-", label="result")
plt.plot(LAmin_groundtruth, "o-", label="ground truth")
plt.plot(LAmin_error, "o-", label="error")
#plt.plot(N_avg_r, "o-", label="result_R")
#plt.plot(N_avg_l, "o-", label="result_L")
plt.xlabel('Soundscapes')
plt.title("Min LA ")
plt.legend()
plt.show()

# 05 percentile LA
plt.figure()
plt.plot(LA05_result, "o-", label="result")
plt.plot(LA05_groundtruth, "o-", label="ground truth")
plt.plot(LA05_error, "o-", label="error")
#plt.plot(N_avg_r, "o-", label="result_R")
#plt.plot(N_avg_l, "o-", label="result_L")
plt.xlabel('Soundscapes')
plt.title("5 percentile LA ")
plt.legend()
plt.show()


# Comparing different non-augmented soundscapes calculation VS ground-truth - LCeq

In [ ]:

[B,A] = C_weighting(fs)

sigL_0001_1_C = lfilter(B,A,sigL_0001_1) #L
sigR_0001_1_C = lfilter(B,A,sigR_0001_1) #R

sigL_0001_2_C = lfilter(B,A,sigL_0001_2) #L
sigR_0001_2_C = lfilter(B,A,sigR_0001_2) #R

sigL_0002_1_C = lfilter(B,A,sigL_0002_1) #L
sigR_0002_1_C = lfilter(B,A,sigR_0002_1) #R

sigL_0002_2_C = lfilter(B,A,sigL_0002_2) #L
sigR_0002_2_C = lfilter(B,A,sigR_0002_2) #R

sigL_0003_1_C = lfilter(B,A,sigL_0003_1) #L
sigR_0003_1_C = lfilter(B,A,sigR_0003_1) #R

sigL_0003_2_C = lfilter(B,A,sigL_0003_2) #L
sigR_0003_2_C = lfilter(B,A,sigR_0003_2) #R

sigL_0004_1_C = lfilter(B,A,sigL_0004_1) #L
sigR_0004_1_C = lfilter(B,A,sigR_0004_1) #R

sigL_0004_2_C = lfilter(B,A,sigL_0004_2) #L
sigR_0004_2_C = lfilter(B,A,sigR_0004_2) #R

sigL_0005_1_C = lfilter(B,A,sigL_0005_1) #L
sigR_0005_1_C = lfilter(B,A,sigR_0005_1) #R

sigL_0005_2_C = lfilter(B,A,sigL_0005_2) #L
sigR_0005_2_C = lfilter(B,A,sigR_0005_2) #R

sigL_0006_1_C = lfilter(B,A,sigL_0006_1) #L
sigR_0006_1_C = lfilter(B,A,sigR_0006_1) #R

sigL_0006_2_C = lfilter(B,A,sigL_0006_2) #L
sigR_0006_2_C = lfilter(B,A,sigR_0006_2) #R

sigL_0007_1_C = lfilter(B,A,sigL_0007_1) #L
sigR_0007_1_C = lfilter(B,A,sigR_0007_1) #R

sigL_0007_2_C = lfilter(B,A,sigL_0007_2) #L
sigR_0007_2_C = lfilter(B,A,sigR_0007_2) #R

sigL_0008_1_C = lfilter(B,A,sigL_0008_1) #L
sigR_0008_1_C = lfilter(B,A,sigR_0008_1) #R

sigL_0008_2_C = lfilter(B,A,sigL_0008_2) #L
sigR_0008_2_C = lfilter(B,A,sigR_0008_2) #R

sigL_0009_1_C = lfilter(B,A,sigL_0009_1) #L
sigR_0009_1_C = lfilter(B,A,sigR_0009_1) #R

sigL_0009_2_C = lfilter(B,A,sigL_0009_2) #L
sigR_0009_2_C = lfilter(B,A,sigR_0009_2) #R

sigL_0010_1_C = lfilter(B,A,sigL_0003_1) #L
sigR_0010_1_C = lfilter(B,A,sigR_0003_1) #R

sigL_0010_2_C = lfilter(B,A,sigL_0010_2) #L
sigR_0010_2_C = lfilter(B,A,sigR_0010_2) #R

In [ ]:
LCeq_groundtruth = np.array([65.58, 67.18, 73.76, 76.74, 64.17, 63.2, 68.28, 68.11, 64.33, 65.73, 64.26, 69.07, 59.34, 59.21, 67.52, 67.71, 69.53, 69.54, 71.68, 71.91])


LCeqL_0001_1 = mean_dB(pressure2leq(sigL_0001_1_C, fs))
LCeqR_0001_1 = mean_dB(pressure2leq(sigR_0001_1_C, fs))
LCeqS_0001_1 =mean_dB([LCeqL_0001_1,LCeqR_0001_1])

LCeqL_0001_2 = mean_dB(pressure2leq(sigL_0001_2_C, fs))
LCeqR_0001_2 = mean_dB(pressure2leq(sigR_0001_2_C, fs))
LCeqS_0001_2 =mean_dB([LCeqL_0001_2,LCeqR_0001_2])

LCeqL_0002_1 = mean_dB(pressure2leq(sigL_0002_1_C, fs))
LCeqR_0002_1 = mean_dB(pressure2leq(sigR_0002_1_C, fs))
LCeqS_0002_1 =mean_dB([LCeqL_0002_1,LCeqR_0002_1])

LCeqL_0002_2 = mean_dB(pressure2leq(sigL_0002_2_C, fs))
LCeqR_0002_2 = mean_dB(pressure2leq(sigR_0002_2_C, fs))
LCeqS_0002_2 =mean_dB([LCeqL_0002_2,LCeqR_0002_2])

LCeqL_0003_1 = mean_dB(pressure2leq(sigL_0003_1_C, fs))
LCeqR_0003_1 = mean_dB(pressure2leq(sigR_0003_1_C, fs))
LCeqS_0003_1 =mean_dB([LCeqL_0003_1,LCeqR_0003_1])

LCeqL_0003_2 = mean_dB(pressure2leq(sigL_0003_2_C, fs))
LCeqR_0003_2 = mean_dB(pressure2leq(sigR_0003_2_C, fs))
LCeqS_0003_2 =mean_dB([LCeqL_0003_2,LCeqR_0003_2])

LCeqL_0004_1 = mean_dB(pressure2leq(sigL_0004_1_C, fs))
LCeqR_0004_1 = mean_dB(pressure2leq(sigR_0004_1_C, fs))
LCeqS_0004_1 =mean_dB([LCeqL_0004_1,LCeqR_0004_1])

LCeqL_0004_2 = mean_dB(pressure2leq(sigL_0004_2_C, fs))
LCeqR_0004_2 = mean_dB(pressure2leq(sigR_0004_2_C, fs))
LCeqS_0004_2 =mean_dB([LCeqL_0004_2,LCeqR_0004_2])

LCeqL_0005_1 = mean_dB(pressure2leq(sigL_0005_1_C, fs))
LCeqR_0005_1 = mean_dB(pressure2leq(sigR_0005_1_C, fs))
LCeqS_0005_1 =mean_dB([LCeqL_0005_1,LCeqR_0005_1])

LCeqL_0005_2 = mean_dB(pressure2leq(sigL_0005_2_C, fs))
LCeqR_0005_2 = mean_dB(pressure2leq(sigR_0005_2_C, fs))
LCeqS_0005_2 =mean_dB([LCeqL_0005_2,LCeqR_0005_2])

LCeqL_0006_1 = mean_dB(pressure2leq(sigL_0006_1_C, fs))
LCeqR_0006_1 = mean_dB(pressure2leq(sigR_0006_1_C, fs))
LCeqS_0006_1 =mean_dB([LCeqL_0006_1,LCeqR_0006_1])

LCeqL_0006_2 = mean_dB(pressure2leq(sigL_0006_2_C, fs))
LCeqR_0006_2 = mean_dB(pressure2leq(sigR_0006_2_C, fs))
LCeqS_0006_2 =mean_dB([LCeqL_0006_2,LCeqR_0006_2])

LCeqL_0007_1 = mean_dB(pressure2leq(sigL_0007_1_C, fs))
LCeqR_0007_1 = mean_dB(pressure2leq(sigR_0007_1_C, fs))
LCeqS_0007_1 =mean_dB([LCeqL_0007_1,LCeqR_0007_1])

LCeqL_0007_2 = mean_dB(pressure2leq(sigL_0007_2_C, fs))
LCeqR_0007_2 = mean_dB(pressure2leq(sigR_0007_2_C, fs))
LCeqS_0007_2 =mean_dB([LCeqL_0007_2,LCeqR_0007_2])

LCeqL_0008_1 = mean_dB(pressure2leq(sigL_0008_1_C, fs))
LCeqR_0008_1 = mean_dB(pressure2leq(sigR_0008_1_C, fs))
LCeqS_0008_1 =mean_dB([LCeqL_0008_1,LCeqR_0008_1])

LCeqL_0008_2 = mean_dB(pressure2leq(sigL_0008_2_C, fs))
LCeqR_0008_2 = mean_dB(pressure2leq(sigR_0008_2_C, fs))
LCeqS_0008_2 =mean_dB([LCeqL_0008_2,LCeqR_0008_2])

LCeqL_0009_1 = mean_dB(pressure2leq(sigL_0009_1_C, fs))
LCeqR_0009_1 = mean_dB(pressure2leq(sigR_0009_1_C, fs))
LCeqS_0009_1 =mean_dB([LCeqL_0009_1,LCeqR_0009_1])

LCeqL_0009_2 = mean_dB(pressure2leq(sigL_0009_2_C, fs))
LCeqR_0009_2 = mean_dB(pressure2leq(sigR_0009_2_C, fs))
LCeqS_0009_2 =mean_dB([LCeqL_0009_2,LCeqR_0009_2])

LCeqL_0010_1 = mean_dB(pressure2leq(sigL_0010_1_C, fs))
LCeqR_0010_1 = mean_dB(pressure2leq(sigR_0010_1_C, fs))
LCeqS_0010_1 =mean_dB([LCeqL_0010_1,LCeqR_0010_1])

LCeqL_0010_2 = mean_dB(pressure2leq(sigL_0010_2_C, fs))
LCeqR_0010_2 = mean_dB(pressure2leq(sigR_0010_2_C, fs))
LCeqS_0010_2 =mean_dB([LCeqL_0010_2,LCeqR_0010_2])

LCeqS_result=np.array([
    LCeqS_0001_1,
    LCeqS_0001_2,
    LCeqS_0002_1,
    LCeqS_0002_2,
    LCeqS_0003_1,
    LCeqS_0003_2,
    LCeqS_0004_1,
    LCeqS_0004_2,
    LCeqS_0005_1,
    LCeqS_0005_2,
    LCeqS_0006_1,
    LCeqS_0006_2,
    LCeqS_0007_1,
    LCeqS_0007_2,
    LCeqS_0008_1,
    LCeqS_0008_2,
    LCeqS_0009_1,
    LCeqS_0009_2,
    LCeqS_0010_1,
    LCeqS_0010_2,
])

LCeq_error=LCeq_groundtruth-LCeqS_result

# Average LCeq
plt.figure()
plt.plot(LCeqS_result, "o-", label="result")
plt.plot(LCeq_groundtruth, "o-", label="ground truth")
plt.plot(LCeq_error, "o-", label="error")

plt.xlabel('Soundscapes')
plt.title("Average LCeq ")
plt.legend()
plt.show()



# Automatizating code

In [ ]:


def _tnr_main_calc(spectrum_db, freq_axis):

    #### Spectrum creation #######################################################


    if len(spectrum_db.shape) == 1:
        nseg = 1
        # Frequency axis of interest
        freq_index = np.where((freq_axis > 20) & (freq_axis < 20000))[0]
        freqs = freq_axis[freq_index]
        spec_db = spectrum_db[freq_index]
        
    elif (len(spectrum_db.shape) > 1) & (len(freq_axis.shape) > 1):
        nseg = spectrum_db.shape[1]
        freqs = [[]for i in range(nseg)]
        spec_db = [[]for i in range(nseg)]
        for i in range(nseg):
            # Frequency axis of interest
            freq_index_rows = np.where((freq_axis[:,i] > 20) & (freq_axis[:,i] < 20000))[0]
            freqs[i] = np.append(freqs[i],freq_axis[freq_index_rows,i])
            spec_db[i] = np.append(spec_db[i],spectrum_db[freq_index_rows,i])
        freqs = np.asarray(freqs)
        spec_db = np.asarray(spec_db)
    
    elif (len(spectrum_db.shape) > 1) & (len(freq_axis.shape) == 1):
        # Frequency axis of interest
        freq_index = np.where((freq_axis > 20) & (freq_axis < 20000))[0]
        # Initialization
        nfreqs = len(freq_index)    
        nseg = spectrum_db.shape[1]
        freqs = np.zeros((nseg,nfreqs))
        spec_db = np.zeros((nseg,nfreqs))
        for i in range(nseg):
            freqs[i,:] = freq_axis[freq_index]
            spec_db[i,:] = spectrum_db[freq_index,i]


    #### Screening to find the potential tonal components ########################

    peak_index = _screening_for_tones(freqs, spec_db, "smoothed", 20, 20000)

    # Initialization of the results lists
    if nseg == 1:
        TNR = []
        t_tnr = []
        tones_freqs = []
        prominence = []
    else:   
        TNR = [[]for i in range(nseg)]
        t_tnr = [[]for i in range(nseg)]
        tones_freqs = [[]for i in range(nseg)]
        prominence = [[]for i in range(nseg)]
    

    #### Evaluation of each candidate ############################################

    for i in range(nseg):
        
        tnr = np.array([], dtype=object)
        
        if nseg == 1:
            peaks = peak_index
            spec = spec_db
            fr = freqs
            frs = freq_axis
        elif nseg > 1:
            peaks = peak_index[i]
            spec = spec_db[i,:]
            fr = freqs[i,:]
            if len(freq_axis.shape)>1:
                frs = freq_axis[:,i]
            else:
                frs = freq_axis
        
        nb_tones = len(peaks)

        # Each candidate is studied and then deleted from the list until all have been treated
        while nb_tones > 0:
            ind = int(peaks[0])
            if len(peaks) > 1:
                ind_p, ind_s, peaks, nb_tones = _find_highest_tone(
                    fr, spec, peaks.astype(int), nb_tones, ind
                )
            else:
                ind_p = ind
                ind_s = None
    
            # multiple tones in a critical band
            if ind_s != None:
                fp = fr[ind_p]
                fs = fr[ind_s]
    
                # proximity criterion
                delta_f = 21 * 10 ** ((1.2 * (np.abs(np.log10(fp / 212))) ** 1.8))
                if np.abs(fs - fp) < delta_f:
    
                    # tone SPL
                    Lp = _peak_level(fr, spec, ind_p)
                    Ls = _peak_level(fr, spec, ind_s)
    
                    Lt = 10 * np.log10(((10 ** (Lp / 10) + 10 ** (Ls / 10))))
    
                    # total SPL in the critical band
                    f1, f2 = _critical_band(fp)
                    low_limit_idx = np.argmin(np.abs(fr - f1))
                    high_limit_idx = np.argmin(np.abs(fr - f2))
    
                    spec_sum = sum(10 ** (spec[low_limit_idx:high_limit_idx] / 10))
                    Ltot = 10 * np.log10(spec_sum)
    
                    # suppression of the second highest tone from the list of tones
                    sup = np.where(peaks == ind_s)[0]
                    peaks = np.delete(peaks, sup)
                    nb_tones -= 1
    
                    delta_ft = 2 * (frs[1] - frs[0])
    
                else:
                    # the two highest tones are not close enough to be considered as one
                    # tone SPL
                    Lt = spec[ind_p]
    
                    # total SPL in the critical band
                    f1, f2 = _critical_band(fr[ind_p])
                    low_limit_idx = np.argmin(np.abs(fr - f1))
                    high_limit_idx = np.argmin(np.abs(fr - f2))
    
                    spec_sum = sum(10 ** (spec[low_limit_idx:high_limit_idx] / 10))
                    Ltot = 10 * np.log10(spec_sum)
    
                    delta_ft = fr[1] - fr[0]
    
            # single tone in a critical band
            else:
                # tone SPL
                Lt = _peak_level(fr, spec, ind_p)
    
                # total SPL in the critical band
                f1, f2 = _critical_band(fr[ind_p])
                low_limit_idx = np.argmin(np.abs(fr - f1))
                high_limit_idx = np.argmin(np.abs(fr - f2))
    
                spec_sum = sum(10 ** (spec[low_limit_idx:high_limit_idx] / 10))
                Ltot = 10 * np.log10(spec_sum)
    
                delta_ft = fr[1] - fr[0]
    
            # SPL of the masking noise
            delta_fc = f2 - f1
            delta_ftot = frs[high_limit_idx] - frs[low_limit_idx]
            Ln = 10 * np.log10(10 ** (Ltot / 10) - 10 ** (Lt / 10)) + 10 * np.log10(
                delta_fc / (delta_ftot - delta_ft)
            )
    
            # Tone-to-noise ratio
            f = fr[ind_p]
            delta_t = Lt - Ln
            if delta_t > 0:
                if nseg > 1:
                    tones_freqs[i] = np.append(tones_freqs[i], f)
                elif nseg == 1:
                    tones_freqs = np.append(tones_freqs, f)
                tnr = np.append(tnr, delta_t)
    
                # Prominence criteria
                if f >= 20 and f < 1000:
                    if delta_t >= 8 + 8.33 * np.log10(1000 / f):
                        if nseg > 1:
                            prominence[i].append(True)                        
                        elif nseg == 1:
                            prominence.append(True)
                    else:
                        if nseg > 1:
                            prominence[i].append(False)
                        
                        elif nseg == 1:
                            prominence.append(False)
                elif f >= 1000 and f <= 20000:
                    if delta_t >= 8:
                        if nseg > 1:
                            prominence[i].append(True)
                        
                        elif nseg == 1:
                            prominence.append(True)
                    else:
                        if nseg > 1:
                            prominence[i].append(False)
                        
                        elif nseg == 1:
                            prominence.append(False)
    
            # suppression from the list of tones
            sup = np.where(peaks == ind_p)[0]
            peaks = np.delete(peaks, sup)
            nb_tones -= 1
    

        if nseg > 1:
            if sum(np.power(10, (tnr[prominence[i]] / 10))) != 0:
                t_tnr[i] = 10 * np.log10(sum(np.power(10, (tnr[prominence[i]] / 10))))
            else:
                t_tnr[i] =  0
            TNR[i] = np.append(TNR[i], tnr)               
                    
                    
        elif nseg == 1:
            if sum(np.power(10, (tnr[prominence] / 10))) != 0:
                t_tnr = np.append(t_tnr,10 * np.log10(sum(np.power(10, (tnr[prominence] / 10)))))
            else:
                t_tnr =  0
            TNR = np.append(TNR, tnr)
        
    tones_freqs = np.asarray(tones_freqs, dtype=object)
    prominence = np.asarray(prominence, dtype=object)

    
    return tones_freqs, TNR , prominence, t_tnr


def tnr_ecma_tv(signal, fs, prominence=False, overlap=0):
    
    if len(signal.shape) == 1:
      
        # Number of points within each frame according to the time resolution of 500ms
        nperseg = int(1 * fs)
        # Overlappinf segment length
        noverlap = int(overlap * nperseg)               
        # Time segmentation of the signal
        sig, time = time_segmentation(signal, fs, nperseg=nperseg, noverlap=noverlap, is_ecma=True)
        # Number of segments
        nseg = sig.shape[1] 
        # Spectrum computation
        spectrum_db, freq_axis = spectrum(sig, fs, db=True)
      
    else:
        nseg = signal.shape[1]
        time = np.linspace(0, signal.shape[0]/fs, num=nseg)
        
        # Compute spectrum
        spectrum_db, freq_axis = spectrum(sig, fs, db=True)
            
            
    # compute tnr values
    tones_freqs, tnr_, prom, t_tnr = _tnr_main_calc(spectrum_db, freq_axis)
 
            
    # Retore the results in a time vs frequency array
    freqs = np.logspace(np.log10(20), np.log10(20000), num=1000)
    tnr = np.zeros((len(freqs), nseg))
    #tnr.fill(np.nan)
    promi = np.empty((len(freqs), nseg), dtype=bool)
    promi.fill(False)
    
    for t in range(nseg):
        for f in range(len(tones_freqs[t])):
            ind = np.argmin(np.abs(freqs - tones_freqs[t][f]))
            if prominence == False:
                tnr[ind, t] = tnr_[t][f]
                promi[ind, t] = prom[t][f]
            if prominence == True:
                if prom[t][f] == True:
                    tnr[ind, t] = tnr_[t][f]
                    promi[ind, t] = prom[t][f]
    t_tnr = np.ravel(t_tnr)

    tnr=(np.sum(tnr, axis=1))

    return t_tnr, tnr, promi, freqs, time     






center_freq = [5.0, 6.3, 8.0, 10.0, 12.5, 16.0, 20.0, 25.0, 31.5, 40.0, 50.0, 63.0, 80.0,  100.0, 125.0, 160.0, 200.0, 250.0, 315.0, 400.0, 500.0, 630.0, 800.0, 1000.0, 1250.0,  1600.0, 2000.0, 2500.0, 3150.0, 4000.0, 5000.0, 6300.0, 8000.0, 10000.0, 12500.0, 16000.0, 20000.0]
            

def freq_limits(f_c):
    f_l=f_c/2**(1/6)
    f_h=f_c*2**(1/6)

    return(f_l, f_h)

def filter_band(magnitude_spectrum, frequency_axis, f_low, f_high):
    # Find indices where frequency is greater than f_low and less than f_high
    indices = np.where((frequency_axis > f_low) & (frequency_axis < f_high))[0]
    
    # Cut the band from both arrays using the indices
    magnitude_spectrum_cut = magnitude_spectrum[indices]
    frequency_axis_cut = frequency_axis[indices]
    
    return magnitude_spectrum_cut, frequency_axis_cut


def fft_hann(t,pt):
    # Signal length
    N = len(pt)
    # Window length (amount of data pts/win)
    n = 8192
    # Overlap
    overlap_ratio=0.5
    overlap=n*overlap_ratio
    num_windows = int((N / n) / (1 - overlap_ratio)) + 1

    # prepare array where each row represents each window and each column the sum of power per band
    results=np.zeros((num_windows,len(center_freq)))
    # Frequencies
    freq = np.fft.fftfreq(n,t[1]-t[0])[0:int(n/2)]
    # Loop over windows
    for i in range(0,num_windows):
        # Select window of complete audio
        begin = int(i*overlap)
        end = int(begin+n)
        signal_cut=pt[begin:end]
        # Fill in with zeros last window if needed
        if(signal_cut.size<n):
            add=n-signal_cut.size
            signal_cut = np.pad(signal_cut, (0, add), mode='constant')
        # Obtain fft
        spectra = np.abs(np.fft.fft(np.hanning(n)*signal_cut))
        spectra=(1/n)*spectra
        spectra=spectra[0:4096] # only positive half
        # Filter band and obtain the power for each band
        for index,f_c in enumerate(center_freq):
            # Calculate frequency band limits
            (f_l, f_h)=freq_limits(f_c)
            # Filter the band
            spectra_cut, freq_cut=filter_band(spectra, freq, f_l, f_h)
            # Save in results the sum of all the power of the band
            if(spectra_cut.shape[0]==0):
                 results[i,index]=0.00002 # 0dB
            else:     
                results[i,index]=np.sum(spectra_cut)
    return 20*np.log10(np.mean(results, axis=0)/0.00002)

In [ ]:
# Load csv with data
data = pd.read_csv("../data/soundscapes.csv")
# Prepare dataframe for new features
column_names = [
    'Savg_s', 'Smax_s', 'Sargmax_s', 'S05_s', 'S10_s', 'S20_s', 'S30_s', 'S40_s', 'S50_s', 'S60_s', 
    'S70_s', 'S80_s', 'S90_s', 'S95_s', 'Navg_s', 'Nrmc_s', 'Nmax_s', 'Nargmax_s', 'N05_s', 'N10_s', 
    'N20_s', 'N30_s', 'N40_s', 'N50_s', 'N60_s', 'N70_s', 'N80_s', 'N90_s', 'N95_s', 'Favg_s', 'Fmax_s', 
    'Fargmax_s', 'F05_s', 'F10_s', 'F20_s', 'F30_s', 'F40_s', 'F50_s', 'F60_s', 'F70_s', 'F80_s', 'F90_s', 
    'F95_s', 'LAavg_s', 'LAmin_s', 'LAargmin_s', 'LAmax_s', 'LAargmax_s', 'LA05_s', 'LA10_s', 'LA20_s', 
    'LA30_s', 'LA40_s', 'LA50_s', 'LA60_s', 'LA70_s', 'LA80_s', 'LA90_s', 'LA95_s', 'LCavg_s', 'LCmin_s', 
    'LCargmin_s', 'LCmax_s', 'LCargmax_s', 'LC05_s', 'LC10_s', 'LC20_s', 'LC30_s', 'LC40_s', 'LC50_s', 
    'LC60_s', 'LC70_s', 'LC80_s', 'LC90_s', 'LC95_s', 'Ravg_s', 'Rmax_s', 'Rargmax_s', 'R05_s', 'R10_s', 
    'R20_s', 'R30_s', 'R40_s', 'R50_s', 'R60_s', 'R70_s', 'R80_s', 'R90_s', 'R95_s', 'Tgavg_s', 'Tavg_s', 
    'Tmax_s', 'Targmax_s', 'T05_s', 'T10_s', 'T20_s', 'T30_s', 'T40_s', 'T50_s', 'T60_s', 'T70_s', 'T80_s', 
    'T90_s', 'T95_s', 'M00005_0_s', 'M00006_3_s', 'M00008_0_s', 'M00010_0_s', 'M00012_5_s', 'M00016_0_s', 
    'M00020_0_s', 'M00025_0_s', 'M00031_5_s', 'M00040_0_s', 'M00050_0_s', 'M00063_0_s', 'M00080_0_s', 
    'M00100_0_s', 'M00125_0_s', 'M00160_0_s', 'M00200_0_s', 'M00250_0_s', 'M00315_0_s', 'M00400_0_s', 
    'M00500_0_s', 'M00630_0_s', 'M00800_0_s', 'M01000_0_s', 'M01250_0_s', 'M01600_0_s', 'M02000_0_s', 
    'M02500_0_s', 'M03150_0_s', 'M04000_0_s', 'M05000_0_s', 'M06300_0_s', 'M08000_0_s', 'M10000_0_s', 
    'M12500_0_s', 'M16000_0_s', 'M20000_0_s'
]
new_df = pd.DataFrame(columns=column_names)
# For each audio...
audios_path = "../data/soundscapes/"
for file in sorted(os.listdir(audios_path)):
        if file.endswith('.mp3') or file.endswith('.wav'):
            # audio path
            audio_path=audios_path+file
            # Find the row where the filename matches in the "soundscape" column
            audio_info = data[data['soundscape'] == file]
            # Get gain
            audio_gain=audio_info['gain_s'].values
            audio_gain=4.6*audio_gain/0.759272
            # Load L and R channels
            sigL, fs = load(audio_path, wav_calib=audio_gain, ch=0) #L
            sigR, fs = load(audio_path, wav_calib=audio_gain, ch=1) #R

            # Calculate loudness
            ## L
            N_l, N_spec_l, bark_axis_l, time_axis_l = loudness_zwtv(sigL, fs, field_type="free")
            # R
            N_r, N_spec_r, bark_axis_r, time_axis_r = loudness_zwtv(sigR, fs, field_type="free")
            ## S
            N_avg_s=(np.mean(N_l)+np.mean(N_r))/2
            N_max_s=(np.max(N_l)+np.max(N_r))/2
            N_05_s=(np.percentile(N_l,95)+np.percentile(N_r,95))/2
            N_10_s=(np.percentile(N_l,90)+np.percentile(N_r,90))/2
            N_20_s=(np.percentile(N_l,80)+np.percentile(N_r,80))/2
            N_30_s=(np.percentile(N_l,70)+np.percentile(N_r,70))/2
            N_40_s=(np.percentile(N_l,60)+np.percentile(N_r,60))/2
            N_50_s=(np.percentile(N_l,50)+np.percentile(N_r,50))/2
            N_60_s=(np.percentile(N_l,40)+np.percentile(N_r,40))/2
            N_70_s=(np.percentile(N_l,30)+np.percentile(N_r,30))/2
            N_80_s=(np.percentile(N_l,20)+np.percentile(N_r,20))/2
            N_90_s=(np.percentile(N_l,10)+np.percentile(N_r,10))/2
            N_95_s=(np.percentile(N_l,5)+np.percentile(N_r,5))/2 

            # Calculate sharpness
            ## L
            S_l, time_axis_l = sharpness_din_tv(sigL, fs, field_type="free", skip=0.5)
            # R
            S_r, time_axis_r = sharpness_din_tv(sigR, fs, field_type="free", skip=0.5)
            ## S
            S_avg_s=(np.mean(S_l)+np.mean(S_r))/2
            S_max_s=(np.max(S_l)+np.max(S_r))/2
            S_05_s=(np.percentile(S_l,95)+np.percentile(S_r,95))/2
            S_10_s=(np.percentile(S_l,90)+np.percentile(S_r,90))/2
            S_20_s=(np.percentile(S_l,80)+np.percentile(S_r,80))/2
            S_30_s=(np.percentile(S_l,70)+np.percentile(S_r,70))/2
            S_40_s=(np.percentile(S_l,60)+np.percentile(S_r,60))/2
            S_50_s=(np.percentile(S_l,50)+np.percentile(S_r,50))/2
            S_60_s=(np.percentile(S_l,40)+np.percentile(S_r,40))/2
            S_70_s=(np.percentile(S_l,30)+np.percentile(S_r,30))/2
            S_80_s=(np.percentile(S_l,20)+np.percentile(S_r,20))/2
            S_90_s=(np.percentile(S_l,10)+np.percentile(S_r,10))/2
            S_95_s=(np.percentile(S_l,5)+np.percentile(S_r,5))/2

            # Calculate LAeqLAeq

            [B_A,A_A] = A_weighting(fs)
            sigL_A = lfilter(B_A,A_A,sigL)
            sigR_A = lfilter(B_A,A_A,sigR)
            LAeq_l=pressure2leq(sigL_A, fs, 0.125)
            LAeq_r=pressure2leq(sigR_A, fs, 0.125)

            LA_avg_s =mean_dB([mean_dB(LAeq_l),mean_dB(LAeq_r)])
            LA_max_s =mean_dB([np.max(LAeq_l),np.max(LAeq_r)])
            LA_min_s =mean_dB([np.min(LAeq_l),np.min(LAeq_r)])
            LA_05_s =mean_dB([np.percentile(LAeq_l,95),np.percentile(LAeq_r,95)])
            LA_10_s =mean_dB([np.percentile(LAeq_l,90),np.percentile(LAeq_r,90)])
            LA_20_s =mean_dB([np.percentile(LAeq_l,80),np.percentile(LAeq_r,80)])
            LA_30_s =mean_dB([np.percentile(LAeq_l,70),np.percentile(LAeq_r,70)])
            LA_40_s =mean_dB([np.percentile(LAeq_l,60),np.percentile(LAeq_r,60)])
            LA_50_s =mean_dB([np.percentile(LAeq_l,50),np.percentile(LAeq_r,50)])
            LA_60_s =mean_dB([np.percentile(LAeq_l,40),np.percentile(LAeq_r,40)])
            LA_70_s =mean_dB([np.percentile(LAeq_l,30),np.percentile(LAeq_r,30)])
            LA_80_s =mean_dB([np.percentile(LAeq_l,20),np.percentile(LAeq_r,20)])
            LA_90_s =mean_dB([np.percentile(LAeq_l,10),np.percentile(LAeq_r,10)])
            LA_95_s =mean_dB([np.percentile(LAeq_l,5),np.percentile(LAeq_r,5)]) 


            # Calculate LCeq
            [B_C,A_C] = C_weighting(fs)
            sigL_C = lfilter(B_C,A_C,sigL)
            sigR_C = lfilter(B_C,A_C,sigR)
            LCeq_l=pressure2leq(sigL_C, fs, 0.125)
            LCeq_r=pressure2leq(sigR_C, fs, 0.125)

            LC_avg_s =mean_dB([mean_dB(LCeq_l),mean_dB(LCeq_r)])
            LC_max_s =mean_dB([np.max(LCeq_l),np.max(LCeq_r)])
            LC_min_s =mean_dB([np.min(LCeq_l),np.min(LCeq_r)])
            LC_05_s =mean_dB([np.percentile(LCeq_l,95),np.percentile(LCeq_r,95)])
            LC_10_s =mean_dB([np.percentile(LCeq_l,90),np.percentile(LCeq_r,90)])
            LC_20_s =mean_dB([np.percentile(LCeq_l,80),np.percentile(LCeq_r,80)])
            LC_30_s =mean_dB([np.percentile(LCeq_l,70),np.percentile(LCeq_r,70)])
            LC_40_s =mean_dB([np.percentile(LCeq_l,60),np.percentile(LCeq_r,60)])
            LC_50_s =mean_dB([np.percentile(LCeq_l,50),np.percentile(LCeq_r,50)])
            LC_60_s =mean_dB([np.percentile(LCeq_l,40),np.percentile(LCeq_r,40)])
            LC_70_s =mean_dB([np.percentile(LCeq_l,30),np.percentile(LCeq_r,30)])
            LC_80_s =mean_dB([np.percentile(LCeq_l,20),np.percentile(LCeq_r,20)])
            LC_90_s =mean_dB([np.percentile(LCeq_l,10),np.percentile(LCeq_r,10)])
            LC_95_s =mean_dB([np.percentile(LCeq_l,5),np.percentile(LCeq_r,5)])

            # Calculate tonality
            _, T_l,_,_,_= tnr_ecma_tv(sigL, fs,prominence=False, overlap=0)
            _, T_r,_,_,_= tnr_ecma_tv(sigR, fs,prominence=False, overlap=0)
            T_avg_s=np.mean((np.mean(T_l)+np.mean(T_r)/2))
            T_max_s=(np.max(T_l)+np.max(T_r))/2
            T_05_s=(np.percentile(T_l,95)+np.percentile(T_r,95))/2
            T_10_s=(np.percentile(T_l,90)+np.percentile(T_r,90))/2
            T_20_s=(np.percentile(T_l,80)+np.percentile(T_r,80))/2
            T_30_s=(np.percentile(T_l,70)+np.percentile(T_r,70))/2
            T_40_s=(np.percentile(T_l,60)+np.percentile(T_r,60))/2
            T_50_s=(np.percentile(T_l,50)+np.percentile(T_r,50))/2
            T_60_s=(np.percentile(T_l,40)+np.percentile(T_r,40))/2
            T_70_s=(np.percentile(T_l,30)+np.percentile(T_r,30))/2
            T_80_s=(np.percentile(T_l,20)+np.percentile(T_r,20))/2
            T_90_s=(np.percentile(T_l,10)+np.percentile(T_r,10))/2
            T_95_s=(np.percentile(T_l,5)+np.percentile(T_r,5))/2

            

            # Calculate M##
            time_step = 1 / fs
            time_vector = np.arange(0, len(sigL) * time_step, time_step)
            M_L=fft_hann(time_vector,sigL_A)
            M_R=fft_hann(time_vector,sigR_A)
            M00005_0_s =mean_dB(np.array([M_L[0],M_R[0]]))
            M00006_3_s = mean_dB(np.array([M_L[1],M_R[1]]))
            M00008_0_s = mean_dB(np.array([M_L[2],M_R[2]]))
            M00010_0_s = mean_dB(np.array([M_L[3],M_R[3]]))
            M00012_5_s = mean_dB(np.array([M_L[4],M_R[4]]))
            M00016_0_s = mean_dB(np.array([M_L[5],M_R[5]]))
            M00020_0_s = mean_dB(np.array([M_L[6],M_R[6]]))
            M00025_0_s = mean_dB(np.array([M_L[7],M_R[7]]))
            M00031_5_s = mean_dB(np.array([M_L[8],M_R[8]]))
            M00040_0_s = mean_dB(np.array([M_L[9],M_R[9]]))
            M00050_0_s = mean_dB(np.array([M_L[10],M_R[10]]))
            M00063_0_s = mean_dB(np.array([M_L[11],M_R[11]]))
            M00080_0_s = mean_dB(np.array([M_L[12],M_R[12]]))
            M00100_0_s = mean_dB(np.array([M_L[13],M_R[13]]))
            M00125_0_s = mean_dB(np.array([M_L[14],M_R[14]]))
            M00160_0_s = mean_dB(np.array([M_L[15],M_R[15]]))
            M00200_0_s = mean_dB(np.array([M_L[16],M_R[16]]))
            M00250_0_s = mean_dB(np.array([M_L[17],M_R[17]]))
            M00315_0_s = mean_dB(np.array([M_L[18],M_R[18]]))
            M00400_0_s = mean_dB(np.array([M_L[19],M_R[19]]))
            M00500_0_s = mean_dB(np.array([M_L[20],M_R[20]]))
            M00630_0_s = mean_dB(np.array([M_L[21],M_R[21]]))
            M00800_0_s = mean_dB(np.array([M_L[22],M_R[22]]))
            M01000_0_s = mean_dB(np.array([M_L[23],M_R[23]]))
            M01250_0_s = mean_dB(np.array([M_L[24],M_R[24]]))
            M01600_0_s = mean_dB(np.array([M_L[25],M_R[25]]))
            M02000_0_s = mean_dB(np.array([M_L[26],M_R[26]]))
            M02500_0_s = mean_dB(np.array([M_L[27],M_R[27]]))
            M03150_0_s = mean_dB(np.array([M_L[28],M_R[28]]))
            M04000_0_s = mean_dB(np.array([M_L[29],M_R[29]]))
            M05000_0_s = mean_dB(np.array([M_L[30],M_R[30]]))
            M06300_0_s = mean_dB(np.array([M_L[31],M_R[31]]))
            M08000_0_s = mean_dB(np.array([M_L[32],M_R[32]]))
            M10000_0_s = mean_dB(np.array([M_L[33],M_R[33]]))
            M12500_0_s = mean_dB(np.array([M_L[34],M_R[34]]))
            M16000_0_s = mean_dB(np.array([M_L[35],M_R[35]]))
            M20000_0_s = mean_dB(np.array([M_L[36],M_R[36]]))
        
            # Calculate roughness

            # Calculate fluctuation strength


            # Add all data to entry
            entry = {
                'soundscape': file,
                'gain': audio_gain,
                'Savg_s': S_avg_s,
                'Srmc_s': 0,
                'Smax_s': S_max_s,
                'Sargmax_s': 0,
                'S05_s': S_05_s,
                'S10_s': S_10_s,
                'S20_s': S_20_s,
                'S30_s': S_30_s,
                'S40_s': S_40_s,
                'S50_s': S_50_s,
                'S60_s': S_60_s,
                'S70_s': S_70_s,
                'S80_s': S_80_s,
                'S90_s': S_90_s,
                'S95_s': S_95_s,
                'Navg_s': N_avg_s,
                'Nrmc_s': 0,
                'Nmax_s': N_max_s,
                'Nargmax_s': 0,
                'N05_s': N_05_s,
                'N10_s': N_10_s,
                'N20_s': N_20_s,
                'N30_s': N_30_s,
                'N40_s': N_40_s,
                'N50_s': N_50_s,
                'N60_s': N_60_s,
                'N70_s': N_70_s,
                'N80_s': N_80_s,
                'N90_s': N_90_s,
                'N95_s': N_95_s,
                'Favg_s': 0,
                'Fmax_s': 0,
                'Fargmax_s': 0,
                'F05_s': 0,
                'F10_s': 0,
                'F20_s': 0,
                'F30_s': 0,
                'F40_s': 0,
                'F50_s': 0,
                'F60_s': 0,
                'F70_s': 0,
                'F80_s': 0,
                'F90_s': 0,
                'F95_s': 0,
                'LAavg_s': LA_avg_s,
                'LAmin_s': LA_min_s,
                'LAargmin_s': 0,
                'LAmax_s': LA_max_s,
                'LAargmax_s': 0,
                'LA05_s': LA_05_s,
                'LA10_s': LA_10_s,
                'LA20_s': LA_20_s,
                'LA30_s': LA_30_s,
                'LA40_s': LA_40_s,
                'LA50_s': LA_50_s,
                'LA60_s': LA_60_s,
                'LA70_s': LA_70_s,
                'LA80_s': LA_80_s,
                'LA90_s': LA_90_s,
                'LA95_s': LA_95_s,
                'LCavg_s': LC_avg_s,
                'LCmin_s': LC_min_s,
                'LCargmin_s': 0,
                'LCmax_s': LC_max_s,
                'LCargmax_s': 0,
                'LC05_s': LC_05_s,
                'LC10_s': LC_10_s,
                'LC20_s': LC_20_s,
                'LC30_s': LC_30_s,
                'LC40_s': LC_40_s,
                'LC50_s': LC_50_s,
                'LC60_s': LC_60_s,
                'LC70_s': LC_70_s,
                'LC80_s': LC_80_s,
                'LC90_s': LC_90_s,
                'LC95_s': LC_95_s,
                'Ravg_s': 0,
                'Rmax_s': 0,
                'Rargmax_s': 0,
                'R05_s': 0,
                'R10_s': 0,
                'R20_s': 0,
                'R30_s': 0,
                'R40_s': 0,
                'R50_s': 0,
                'R60_s': 0,
                'R70_s': 0,
                'R80_s': 0,
                'R90_s': 0,
                'R95_s': 0,
                'Tgavg_s': 0,
                'Tavg_s': T_avg_s,
                'Tmax_s': T_max_s,
                'Targmax_s': 0,
                'T05_s': T_05_s,
                'T10_s': T_10_s,
                'T20_s': T_20_s,
                'T30_s': T_30_s,
                'T40_s': T_40_s,
                'T50_s': T_50_s,
                'T60_s': T_60_s,
                'T70_s': T_70_s,
                'T80_s': T_80_s,
                'T90_s': T_90_s,
                'T95_s': T_95_s,
                'M00005_0_s': M00005_0_s,
                'M00006_3_s': M00006_3_s,
                'M00008_0_s': M00008_0_s,
                'M00010_0_s': M00010_0_s,
                'M00012_5_s': M00012_5_s,
                'M00016_0_s': M00016_0_s,
                'M00020_0_s': M00020_0_s,
                'M00025_0_s': M00025_0_s,
                'M00031_5_s': M00031_5_s,
                'M00040_0_s': M00040_0_s,
                'M00050_0_s': M00050_0_s,
                'M00063_0_s': M00063_0_s,
                'M00080_0_s': M00080_0_s,
                'M00100_0_s': M00100_0_s,
                'M00125_0_s': M00125_0_s,
                'M00160_0_s': M00160_0_s,
                'M00200_0_s': M00200_0_s,
                'M00250_0_s': M00250_0_s,
                'M00315_0_s': M00315_0_s,
                'M00400_0_s': M00400_0_s,
                'M00500_0_s': M00500_0_s,
                'M00630_0_s': M00630_0_s,
                'M00800_0_s': M00800_0_s,
                'M01000_0_s': M01000_0_s,
                'M01250_0_s': M01250_0_s,
                'M01600_0_s': M01600_0_s,
                'M02000_0_s': M02000_0_s,
                'M02500_0_s': M02500_0_s,
                'M03150_0_s': M03150_0_s,
                'M04000_0_s': M04000_0_s,
                'M05000_0_s': M05000_0_s,
                'M06300_0_s': M06300_0_s,
                'M08000_0_s': M08000_0_s,
                'M10000_0_s': M10000_0_s,
                'M12500_0_s': M12500_0_s,
                'M16000_0_s': M16000_0_s,
                'M20000_0_s': M20000_0_s,
            }

            new_df = pd.concat([new_df, pd.DataFrame([entry])], ignore_index=True)

            print("File done ", file)

            




In [ ]:
# Plot
for feature in column_names:
    vector=np.array(new_df[feature])
    if((feature!="soundscape" or feature!="gain")):             
        plt.figure()
        if(type(data[feature][0])==tuple):
            x_values = [x[0] for x in data[feature][0:len(new_df[feature])]]
            plt.plot(x_values, "o-", label="ground truth")
        else:
            plt.plot(data[feature][0:len(new_df[feature])], "o-", label="ground truth")
        plt.plot(new_df[feature], "o-", label="result")
        plt.xlabel('Soundscapes')
        plt.title(feature)
        plt.legend()
        plt.show() 

In [ ]:
new_df.to_csv('generated.csv', index=False)

In [ ]:
def freq_limits(f_c):
    f_l=f_c/2**(1/6)
    f_h=f_c*2**(1/6)

    return(f_l, f_h)

def filter_band(magnitude_spectrum, frequency_axis, f_low, f_high):
    # Find indices where frequency is greater than f_low and less than f_high
    indices = np.where((frequency_axis > f_low) & (frequency_axis < f_high))[0]
    
    # Cut the band from both arrays using the indices
    magnitude_spectrum_cut = magnitude_spectrum[indices]
    frequency_axis_cut = frequency_axis[indices]
    
    return magnitude_spectrum_cut, frequency_axis_cut


def fft_hann(t,pt):
    # Signal length
    N = len(pt)
    # Window length (amount of data pts/win)
    n = 8192
    # Overlap
    overlap_ratio=0.5
    overlap=n*overlap_ratio
    num_windows = int((N / n) / (1 - overlap_ratio)) + 1

    # prepare array where each row represents each window and each column the sum of power per band
    results=np.zeros((num_windows,len(center_freq)))
    # Frequencies
    freq = np.fft.fftfreq(n,t[1]-t[0])[0:int(n/2)]
    # Loop over windows
    for i in range(0,num_windows):
        # Select window of complete audio
        begin = int(i*overlap)
        end = int(begin+n)
        signal_cut=pt[begin:end]
        # Fill in with zeros last window if needed
        if(signal_cut.size<n):
            add=n-signal_cut.size
            signal_cut = np.pad(signal_cut, (0, add), mode='constant')
        # Obtain fft
        spectra = np.abs(np.fft.fft(np.hanning(n)*signal_cut))
        spectra=(1/n)*spectra
        spectra=spectra[0:4096] # only positive half
        # Filter band and obtain the power for each band
        for index,f_c in enumerate(center_freq):
            # Calculate frequency band limits
            (f_l, f_h)=freq_limits(f_c)
            print("central freq ", f_c, " has limits ",(f_l, f_h))
            # Filter the band
            spectra_cut, freq_cut=filter_band(spectra, freq, f_l, f_h)
            print(" number of points in band ", spectra_cut.shape, " when original was ", spectra.shape )
            # Save in results the sum of all the power of the band
            if(spectra_cut.shape[0]==0):
                 results[i,index]=0.00002 # 0dB
            else:     
                results[i,index]=np.sum(spectra_cut)
    return 20*np.log10(np.mean(results, axis=0)/0.00002)


center_freq = [5.0, 6.3, 8.0, 10.0, 12.5, 16.0, 20.0, 25.0, 31.5, 40.0, 50.0, 63.0, 80.0,  100.0, 125.0, 160.0, 200.0, 250.0, 315.0, 400.0, 500.0, 630.0, 800.0, 1000.0, 1250.0,  1600.0, 2000.0, 2500.0, 3150.0, 4000.0, 5000.0, 6300.0, 8000.0, 10000.0, 12500.0, 16000.0, 20000.0]
            
audios_path = "../data/soundscapes/"
for file in sorted(os.listdir(audios_path)):
        if file.endswith('.mp3') or file.endswith('.wav'):
            
            # audio path
            audio_path=audios_path+file
            # Find the row where the filename matches in the "soundscape" column
            audio_info = data[data['soundscape'] == file]
            # Get gain
            audio_gain=audio_info['gain_s'].values
            audio_gain=4.6*audio_gain/0.759272
            # Load L and R channels
            sigL, fs = load(audio_path, wav_calib=audio_gain, ch=0) #L
            sigL_A = lfilter(B_A,A_A,sigL)
            sigR, fs = load(audio_path, wav_calib=audio_gain, ch=1) 
            sigR_A = lfilter(B_A,A_A,sigR)
            print(file)
# Calculate the time step (duration of each sample)
time_step = 1 / fs
# Create the time vector based on the duration of the signal
time_vector = np.arange(0, len(sigL) * time_step, time_step)
result=fft_hann(time_vector,sigL_A)
print(mean_dB(result, axis=1))
result=fft_hann(time_vector,sigR_A)



# Prueba

In [ ]:
audios_path = "../data/soundscapes_augmented/"
file="R0001_segment_binaural_44100_1.wav"
file="fold_2_participant_00002_stimulus_20.wav"
audio_path=audios_path+file
# Find the row where the filename matches in the "soundscape" column
# Get gain
audio_gain=5.661568894352099
#audio_gain=4.6
# Load L and R channels
sigL, fs = load(audio_path, wav_calib=audio_gain, ch=0) #L
sigR, fs = load(audio_path, wav_calib=audio_gain, ch=1) #R
#pressure to Leq
sigL_eq=mean_dB(pressure2leq(sigL, fs))
sigR_eq=mean_dB(pressure2leq(sigR, fs))
sigS_eq=mean_dB([sigL_eq,sigR_eq])

[B_A,A_A] = A_weighting(fs)
sigL_A = lfilter(B_A,A_A,sigL)
sigR_A = lfilter(B_A,A_A,sigR)
LAeq_r=pressure2leq(sigL_A, fs)
sigL_Aeq=mean_dB(pressure2leq(sigL_A, fs))
sigR_Aeq=mean_dB(pressure2leq(sigR_A, fs))
sigS_Aeq=mean_dB([sigL_Aeq,sigR_Aeq])
LA_max_s =np.max(LAeq_r)
LA_min_s =np.min(LAeq_r)
LA_05_s =np.percentile(LAeq_r,95)
LA_10_s =np.percentile(LAeq_r,90)
LA_20_s =np.percentile(LAeq_r,80)
LA_30_s =np.percentile(LAeq_r,70)
LA_40_s =np.percentile(LAeq_r,60)
LA_50_s =np.percentile(LAeq_r,50)
LA_60_s =np.percentile(LAeq_r,40)
LA_70_s =np.percentile(LAeq_r,30)
LA_80_s =np.percentile(LAeq_r,20)
LA_90_s =np.percentile(LAeq_r,10)
LA_95_s =np.percentile(LAeq_r,5)
print("LAeq avg= ", sigS_Aeq ,LA_max_s, LA_min_s, LA_05_s, LA_10_s, LA_20_s, LA_30_s,LA_40_s, LA_50_s, LA_60_s, LA_70_s, LA_80_s ,LA_90_s, LA_95_s)

print(sigS_eq,sigL_eq, sigR_eq)
trueLeq=68.0358148#69.033371 #65.666663
difference=trueLeq-sigR_eq
factor=10**(difference/20)
print(factor)